In [1]:
%%time
dataset_name = "W19_comb"
df_list = [ "BES_Panel" ]

%matplotlib inline 
%run BES_header.py {dataset_name} {df_list}

if "id" in BES_Panel.columns:
    BES_Panel = BES_Panel.set_index("id").sort_index()

(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)    
    
# get full set of inferred "cross wave" auth-lib/left-right values and ages
pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')



Toggle code

var_type (7911, 14)
Wall time: 27.7 s


Media variables
all these variables reflect the "profile_newspaper_readership_201" yougov variable

note that this is "Which daily newspaper do you read most often?"

you could legitimately read the Express and Mail about as often and oscillate between

also there's a huge editorial (and anecdotally I understand readership) difference between

Mail on Sunday (Remain! - contains weekly TV listings) and the Daily Mail (Leave!)

"profiles_newspaper2W16" is obviously just a "top up" sample

"profile_newspaper" is an amalgamation of all the waves

Full Waves 10, 11, 12, 13, 15

In [2]:
search(BES_Panel,"newspaper")

newspaperReadW15          30842
profile_newspaperW10      30237
profile_newspaperW11      30956
profile_newspaper2W12     34394
profile_newspaperW13      31136
profiles_newspaper2W16    13775
profiles_newspaper2W17    34366
profile_newspaper         56968
dtype: int64

In [3]:
BES_Panel[search(BES_Panel,"newspaper").index[:-1]]

,newspaperReadW15,profile_newspaperW10,profile_newspaperW11,profile_newspaper2W12,profile_newspaperW13,profiles_newspaper2W16,profiles_newspaper2W17
id,,,,,,,
1,NaN,None,NaN,NaN,NaN,NaN,NaN
2,NaN,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,The Daily Telegraph,NaN,The Daily Telegraph
3,None,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,The Mirror / Daily Record,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,The Guardian
98448,NaN,NaN,NaN,NaN,NaN,NaN,None
98449,NaN,NaN,NaN,NaN,NaN,NaN,None


In [7]:
BES_Panel[search(BES_Panel,"newspaper").index].apply(lambda x: "|".join(x.cat.categories)).value_counts()
# consistent categories

The Express|The Daily Mail / The Scottish Daily Mail|The Mirror / Daily Record|The Daily Star / The Daily Star of Scotland|The Sun|The Daily Telegraph|The Financial Times|The Guardian|The Independent|The Times|The Scotsman|The Herald (Glasgow)|The Western Mail|Other local daily morning newspaper|Other Newspaper|None    8
dtype: int64

In [9]:
BES_Panel["profile_newspaper"].value_counts()

None                                           17415
The Sun                                         7420
The Daily Mail / The Scottish Daily Mail        7390
The Guardian                                    4461
The Mirror / Daily Record                       4407
Other Newspaper                                 3743
The Daily Telegraph                             2726
The Times                                       2720
Other local daily morning newspaper             2265
The Independent                                 1339
The Express                                     1068
The Herald (Glasgow)                             562
The Daily Star / The Daily Star of Scotland      539
The Scotsman                                     391
The Financial Times                              319
The Western Mail                                 203
Name: profile_newspaper, dtype: int64

In [30]:
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=False,match=False,
                                   low_priority_values=["Other local daily morning newspaper","Other Newspaper","None"])

Amalgamating variables: 
{10: 'profile_newspaperW10', 11: 'profile_newspaperW11', 12: 'profile_newspaper2W12', 13: 'profile_newspaperW13', 16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profile_newspaperW10', 'profile_newspaperW11', 'profile_newspaper2W12', 'profile_newspaperW13', 'profiles_newspaper2W16', 'profiles_newspaper2W17']


In [33]:
newspaper_amalg.value_counts()

None                                           28027
The Daily Mail / The Scottish Daily Mail        9331
The Guardian                                    7430
The Sun                                         4906
The Times                                       3458
The Daily Telegraph                             3230
Other Newspaper                                 3095
The Mirror / Daily Record                       2853
The Independent                                 1969
Other local daily morning newspaper             1928
The Express                                     1297
The Financial Times                              502
The Daily Star / The Daily Star of Scotland      422
The Herald (Glasgow)                             363
The Scotsman                                     250
The Western Mail                                 140
Name: profile_newspaperW10&11&12&13&16&17, dtype: int64

In [34]:
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=False,match=False,
                                   low_priority_values=[])
newspaper_amalg.value_counts()

Amalgamating variables: 
{10: 'profile_newspaperW10', 11: 'profile_newspaperW11', 12: 'profile_newspaper2W12', 13: 'profile_newspaperW13', 16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profile_newspaperW10', 'profile_newspaperW11', 'profile_newspaper2W12', 'profile_newspaperW13', 'profiles_newspaper2W16', 'profiles_newspaper2W17']


None                                           30597
The Daily Mail / The Scottish Daily Mail        8518
The Guardian                                    6616
The Sun                                         4524
Other Newspaper                                 3727
The Times                                       3149
The Daily Telegraph                             2958
The Mirror / Daily Record                       2584
Other local daily morning newspaper             2304
The Independent                                 1595
The Express                                     1159
The Financial Times                              447
The Daily Star / The Daily Star of Scotland      387
The Herald (Glasgow)                             299
The Scotsman                                     220
The Western Mail                                 117
Name: profile_newspaperW10&11&12&13&16&17, dtype: int64

In [51]:
%%time
newspaper_amalg = amalgamate_waves(BES_Panel,"newspaper2?W",forward_fill=True,match=False,
                                   low_priority_values=[])
newspaper_amalg.value_counts()

Amalgamating variables: 
{10: 'profile_newspaperW10', 11: 'profile_newspaperW11', 12: 'profile_newspaper2W12', 13: 'profile_newspaperW13', 16: 'profiles_newspaper2W16', 17: 'profiles_newspaper2W17'} ['profile_newspaperW10', 'profile_newspaperW11', 'profile_newspaper2W12', 'profile_newspaperW13', 'profiles_newspaper2W16', 'profiles_newspaper2W17']
Wall time: 1min 40s


None                                           32368
The Daily Mail / The Scottish Daily Mail        8043
The Guardian                                    6833
The Sun                                         4074
Other Newspaper                                 3423
The Times                                       3195
The Daily Telegraph                             2848
The Mirror / Daily Record                       2268
Other local daily morning newspaper             2090
The Independent                                 1593
The Express                                     1099
The Financial Times                              478
The Daily Star / The Daily Star of Scotland      312
The Herald (Glasgow)                             284
The Scotsman                                     189
The Western Mail                                 104
Name: profile_newspaperW10&11&12&13&16&17, dtype: int64

In [ ]:
# they're a bit None heavy (and region matters)

In [4]:
BES_Panel["profile_newspaper"].value_counts()
### hmm - discrepancy alert

None                                           17415
The Sun                                         7420
The Daily Mail / The Scottish Daily Mail        7390
The Guardian                                    4461
The Mirror / Daily Record                       4407
Other Newspaper                                 3743
The Daily Telegraph                             2726
The Times                                       2720
Other local daily morning newspaper             2265
The Independent                                 1339
The Express                                     1068
The Herald (Glasgow)                             562
The Daily Star / The Daily Star of Scotland      539
The Scotsman                                     391
The Financial Times                              319
The Western Mail                                 203
Name: profile_newspaper, dtype: int64

In [35]:
search(BES_Panel,"newspaper").index

Index(['newspaperReadW15', 'profile_newspaperW10', 'profile_newspaperW11',
       'profile_newspaper2W12', 'profile_newspaperW13',
       'profiles_newspaper2W16', 'profiles_newspaper2W17',
       'profile_newspaper'],
      dtype='object')

In [36]:
BES_Panel["newspaperReadW15"].value_counts()


None                                           15350
The Daily Mail / The Scottish Daily Mail        3715
The Guardian                                    2617
The Sun                                         1804
The Times                                       1415
Other Newspaper                                 1292
The Daily Telegraph                             1187
The Mirror / Daily Record                        956
Other local daily morning newspaper              843
The Independent                                  619
The Express                                      507
The Financial Times                              189
The Daily Star / The Daily Star of Scotland      140
The Herald (Glasgow)                              93
The Scotsman                                      72
The Western Mail                                  43
Name: newspaperReadW15, dtype: int64

In [37]:
BES_Panel["profile_newspaperW10"].value_counts()

None                                           12611
The Daily Mail / The Scottish Daily Mail        4117
The Guardian                                    2461
The Sun                                         2334
Other Newspaper                                 1798
The Mirror / Daily Record                       1403
The Times                                       1358
The Daily Telegraph                             1285
Other local daily morning newspaper             1127
The Independent                                  547
The Express                                      524
The Daily Star / The Daily Star of Scotland      212
The Financial Times                              152
The Herald (Glasgow)                             144
The Scotsman                                     106
The Western Mail                                  58
Name: profile_newspaperW10, dtype: int64

In [38]:
BES_Panel["profile_newspaperW11"].value_counts()

None                                           12990
The Daily Mail / The Scottish Daily Mail        4147
The Guardian                                    2606
The Sun                                         2318
Other Newspaper                                 1906
The Times                                       1402
The Mirror / Daily Record                       1373
The Daily Telegraph                             1342
Other local daily morning newspaper             1062
The Express                                      587
The Independent                                  542
The Daily Star / The Daily Star of Scotland      200
The Herald (Glasgow)                             162
The Financial Times                              152
The Scotsman                                     105
The Western Mail                                  62
Name: profile_newspaperW11, dtype: int64

In [40]:
BES_Panel["profile_newspaper2W12"].value_counts()

None                                           13987
The Daily Mail / The Scottish Daily Mail        4330
The Guardian                                    3129
Other Newspaper                                 2704
The Sun                                         2182
The Times                                       1709
The Daily Telegraph                             1599
The Mirror / Daily Record                       1349
Other local daily morning newspaper             1180
The Independent                                  788
The Express                                      661
The Daily Star / The Daily Star of Scotland      202
The Financial Times                              200
The Herald (Glasgow)                             199
The Scotsman                                     111
The Western Mail                                  64
Name: profile_newspaper2W12, dtype: int64

In [41]:
BES_Panel["profile_newspaperW13"].value_counts()

None                                           13111
The Daily Mail / The Scottish Daily Mail        4209
The Guardian                                    2617
The Sun                                         2237
Other Newspaper                                 1914
The Times                                       1434
The Daily Telegraph                             1389
The Mirror / Daily Record                       1378
Other local daily morning newspaper             1058
The Express                                      592
The Independent                                  540
The Daily Star / The Daily Star of Scotland      201
The Herald (Glasgow)                             159
The Financial Times                              138
The Scotsman                                     106
The Western Mail                                  53
Name: profile_newspaperW13, dtype: int64

In [43]:
BES_Panel["profiles_newspaper2W16"].value_counts()

None                                           6805
The Daily Mail / The Scottish Daily Mail       1561
The Guardian                                   1535
The Sun                                         630
The Daily Telegraph                             618
The Times                                       607
Other Newspaper                                 533
The Independent                                 373
Other local daily morning newspaper             366
The Mirror / Daily Record                       296
The Express                                     215
The Financial Times                             110
The Daily Star / The Daily Star of Scotland      51
The Scotsman                                     32
The Herald (Glasgow)                             30
The Western Mail                                 13
Name: profiles_newspaper2W16, dtype: int64

In [44]:
BES_Panel["profiles_newspaper2W17"].value_counts()

None                                           17703
The Daily Mail / The Scottish Daily Mail        3908
The Guardian                                    3136
The Sun                                         1853
The Times                                       1512
Other Newspaper                                 1336
The Daily Telegraph                             1302
The Mirror / Daily Record                        955
Other local daily morning newspaper              913
The Independent                                  658
The Express                                      529
The Financial Times                              219
The Daily Star / The Daily Star of Scotland      123
The Herald (Glasgow)                             107
The Scotsman                                      73
The Western Mail                                  39
Name: profiles_newspaper2W17, dtype: int64

In [56]:
(BES_Panel[search(BES_Panel,"newspaper2?W").index]=="The Sun").any(axis=1).sum()

## how come 2000 extra Sun readers just appear for profile_newspaper?

5254

In [48]:

BES_Panel[search(BES_Panel,"newspaper2?W").index].loc[BES_Panel["profile_newspaper"]=="The Sun"].dropna(axis=0,how='all')

,profile_newspaperW10,profile_newspaperW11,profile_newspaper2W12,profile_newspaperW13,profiles_newspaper2W16,profiles_newspaper2W17
id,,,,,,
73,NaN,NaN,Other local daily morning newspaper,Other local daily morning newspaper,NaN,NaN
129,NaN,NaN,The Sun,The Sun,NaN,NaN
215,NaN,NaN,NaN,NaN,NaN,The Sun
228,None,The Sun,Other local daily morning newspaper,NaN,NaN,NaN
232,The Daily Star / The Daily Star of Scotland,The Daily Star / The Daily Star of Scotland,The Daily Star / The Daily Star of Scotland,The Daily Star / The Daily Star of Scotland,NaN,The Daily Star / The Daily Star of Scotland
...,...,...,...,...,...,...
57356,The Sun,The Sun,The Sun,The Sun,NaN,NaN
57464,The Sun,The Sun,None,The Sun,NaN,NaN
57468,The Sun,The Sun,The Mirror / Daily Record,The Sun,NaN,The Sun


In [50]:
BES_Panel["profiles_newspaper2W17"][BES_Panel["profile_newspaper"]=="The Sun"].value_counts()

The Sun                                        911
None                                           846
The Daily Mail / The Scottish Daily Mail       148
Other Newspaper                                 43
Other local daily morning newspaper             42
The Mirror / Daily Record                       35
The Guardian                                    27
The Times                                       18
The Daily Star / The Daily Star of Scotland     18
The Express                                     15
The Daily Telegraph                             14
The Financial Times                              6
The Independent                                  3
The Herald (Glasgow)                             2
The Western Mail                                 0
The Scotsman                                     0
Name: profiles_newspaper2W17, dtype: int64

In [ ]:
BES Media module variables
infoSource

During the last seven days, on average how much time (if any) have you spent per day following news about

politics or current affairs from each of these sources?

TV/Paper/Radio/Internet/People

Full waves 4, 5, 6, 7, 8, 12, 13, 15

In [824]:
search(BES_Panel,"infoSource")

infoSourceTVW4           31552
infoSourcePaperW4        31552
infoSourceRadioW4        31552
infoSourceInternetW4     31552
infoSourcePeopleW4       31552
infoSourceTVW5           30969
infoSourcePaperW5        30969
infoSourceRadioW5        30969
infoSourceInternetW5     30969
infoSourcePeopleW5       30969
infoSourceTVW6           30257
infoSourcePaperW6        30257
infoSourceRadioW6        30257
infoSourceInternetW6     30257
infoSourcePeopleW6       30257
infoSourceTVW7           30895
infoSourcePaperW7        30895
infoSourceRadioW7        30895
infoSourceInternetW7     30895
infoSourcePeopleW7       30895
infoSourceTVW8           33502
infoSourcePaperW8        33502
infoSourceRadioW8        33502
infoSourceInternetW8     33502
infoSourcePeopleW8       33502
infoSourceTVW12          34394
infoSourcePaperW12       34394
infoSourceRadioW12       34394
infoSourceInternetW12    34394
infoSourceTVW13          31136
infoSourcePaperW13       31136
infoSourceRadioW13       31136
infoSour

In [5]:
BES_Panel["infoSourceTVW4"].value_counts()

Less than 1/2 hour      8861
1/2 hour to 1 hour      8585
1 to 2 hours            5334
None, no time at all    4199
More than 2 hours       3505
Don't know              1068
Name: infoSourceTVW4, dtype: int64

In [ ]:
twitterUse: Full Waves 4, 5, 6, 12

Do you use Twitter? Yes/No/DK (mostly No!, ~8000/30+k = Yes)

~8000 people that say Yes then asked

twitterInfo(_1/2/3) Partial Sample Waves 4, 5, 12

In the last 4 weeks, have you read news or found information about the upcoming general election or politics

more generally that was tweeted by any of the following people or organisations? Tick all that apply

twitterInfo_1: Candidates or parties on Twitter

twitterInfo_2: Someone you know personally on Twitter

twitterInfo_3: Other people on Twitter e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on twitter (mostly Don't follow politics on twitter)

twitterInfof2f Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through twitter?
Yes/No/DK

In [825]:
search(BES_Panel,"twitter")

twitterUseW4        31551
twitterInfo_1W4      8799
twitterInfo_2W4      8799
twitterInfo_3W4      8799
twitterUseW5        30970
twitterInfo_1W5      8777
twitterInfo_2W5      8777
twitterInfo_3W5      8777
twitterUseW6        30257
twitterInfof2fW6     8440
twitterUseW12       34394
twitterInfo_1W12     9848
twitterInfo_2W12     9848
twitterInfo_3W12     9848
twitterUseW18       37825
twitterInfo_1W18    10807
twitterInfo_2W18    10807
twitterInfo_3W18    10807
dtype: int64

In [6]:
BES_Panel["twitterInfof2fW6"].value_counts()

Yes           4505
No            3821
Don't know     114
Name: twitterInfof2fW6, dtype: int64

In [ ]:
fbUse: Full Waves 4, 5, 6, 12

Do you use Facebook? Yes/No/DK (mostly No!, ~20k/30+k = Yes)

~20000 people that say Yes then asked

fbInfo(_1/2/3) Partial Sample Waves 4, 5, 12

In the last 4 weeks, have you read news or found information about the upcoming general election or politics more

generally that was posted on Facebook by any of the following people or organisations? Tick all that apply

fbInfo_1 Candidates or parties on Facebook

fbInfo_2 Someone you know personally on Facebook

fbInfo_3 Other people on Facebook e.g. commentators, journalists, activists

Yes/No/DK/Don't follow politics on facebook (mostly Don't follow politics on facebook - even moreso than twitter)

fbInfof2fW6 Partial Sample Wave 6
During the election campaign, did you get news about politics and the election through Facebook?
Yes/No/DK

In [827]:
search(BES_Panel,"fb",case_sensitive=True)

fbUseW4        31551
fbInfo_1W4     21008
fbInfo_2W4     21008
fbInfo_3W4     21008
fbUseW5        30970
fbInfo_1W5     20632
fbInfo_2W5     20632
fbInfo_3W5     20632
fbUseW6        30258
fbInfof2fW6    20233
fbUseW12       34394
fbInfo_1W12    23538
fbInfo_2W12    23538
fbInfo_3W12    23538
fbUseW18       37825
fbInfo_1W18    25248
fbInfo_2W18    25248
fbInfo_3W18    25248
dtype: int64

In [7]:
BES_Panel["fbInfo_1W4"].value_counts()

Don't follow politics on facebook    14358
No                                    3038
Yes                                   2609
Don't know                            1003
Name: fbInfo_1W4, dtype: int64

In [ ]:
euSources Full Waves 7, 8

Have you heard anything about the EU referendum from any of these during the last seven days?

Please tick all that apply.

euSources_1: Television

euSources_2: Newspaper (including online)

euSources_3: Radio

euSources_4: Internet (not including online newspapers)

euSources_5: Talking to other people

Yes/No/DK

In [828]:
search(BES_Panel,"euSources")

euSources_1W7      30895
euSources_2W7      30895
euSources_3W7      30895
euSources_4W7      30895
euSources_5W7      30895
euSources_111W7    30781
euSources_1W8      33502
euSources_2W8      33502
euSources_3W8      33502
euSources_4W8      33502
euSources_5W8      33502
euSources_111W8    33391
dtype: int64

In [829]:
BES_Panel["euSources_1W7"].value_counts()

Yes           19147
No             9568
Don't know     2180
Name: euSources_1W7, dtype: int64

need to download/open up the open-ended question dataset for
tvWatch/radioList/paperRead/internetRead

urk - ugly human data

What are the most important Internet sources (including online newspapers) you read

for information about politics and the election?

List up to three websites or click next if there are no websites you can think of.

TV/radio/newspaper/internet

whole bunch of variables about visiting a party site/sharing party website/fb/twitter stuff -> ignoring

In [832]:
df=pd.read_pickle('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.zip', compression='zip')

In [833]:
crap_list = ["__na__","none","dk","__dk__","na","0","?",".","don't know","dont know","-","d/k","n/a","o","n",
             "x","dn","no","n.a","a","not sure","1","2","3","4","5","6","7","8","9","non","nil","no tv","no idea",
             "unsure","s","/",","," none","n/c","don't read newspapers","n a","nothing","h","g","any",
             "i don't read newspapers","f","e","all","c","??","z","l","v","???","q","b","i don't know","idk","do not read",
             "pass","various","m","do not know","an","as above","..","not applicable","d",",",
             'k', "'", '\\', 'y', 'w', '"', '=', 'r', 'j', 'u', 't', 'p', '`', ';', '>', '+', '!',
             '--','00','.?','//',',,',"''",'>?','?/','@?',"'i",'p<','/.',".'",'>/',',l','n/',':?',' 1',
             '...','///','---','...','n\\a','n?a','n,a','000',
             'n/a.','....','////','????','i dk','any.','n./a','n.,a',' n/a','`n.a','..?.','`n/a','in/a','-///',
             'n.aq','n/a/','----','n /a','nm/a',
             '?????','no ne','n.?/a','-----','.....','/////','none-','nione','/////','n.a..','.none','noene', ',,,,,',
             "can't",'non e', '>>>>>', "do 't",'"non"','nmone',
             "don't",'?????', 'nnone', 'none3', 'onone', 'nonet','n.?/a','none`','arsed','noned','-----','.....',
             'nnoen','none-','nione','/////','n.a..','noner',',,,,,',"can't",'non e','>>>>>',"do 't",'"non"','nmone',
             'xxxxxxx','xxxxxx','xxxxx','xxxx','xxx','xx','x',
             "i don't",'notsure','any/all','did not','noneone','i avoid',
             "i didn't",'whatever', 'no telly', 'i do not', 'your mum','dontread','read the', 'dont use',
             'fkuyhhjh','xcvxcvxc','kjgmfdgf','dddssdsd','jgcvhbmn','xcvcxvxc','tyyuhjkl','wsrfgwrg','ghhhgfgj',
             'hjfcgchg', 'ahsjsjsj','dontwatch','nonr','dont','d k','none','i dont','no e','noine','noe',"i haven't", '"',
             'ditto','dont read','jjj', 'iii', 'jj','djajsjajs','jyfjhjm','don tknow',
             'not sure', 'vznfndndk', 'no paper','sss',"don't kbo",'sdvsdf','in/a','ghgfgg',';iluluyfg','gcfhd',
             'iiii',"don't kno",'not known',"don't kno", 'na',
             'scvxdv','czdvfb', 'rghwrghwr','shsjjs','chifyj', 'xfckkj','fhhjkkkkk','ghsgfshsf','wrgwrhg','i didnt','werhwrhwr',
             'i have no tv','"', 'u', 'n', 'd',
             'do not watch tv',"don't watch","no radio","don't listen to radio","none","nobe","same",
             "radio","can't remember","next","nk","dont listen to the radio","good","npne",
             "not one","know","listen","noen","do not listen to radio","no other","do not read newspapers","nine",
             "dont read newspapers","varies","nill","no newspaper","there is no trustworthy newspaper","i do not read newspapers",
             "no scooby doo","not","all are biased","nothing specific","hey","not read","stop","lies lies lies on waste paper","out of uk at present",
             "never","away on hol","did not read","i don not read politics in newspapers","havent read a newspaper","don't read","n/a don't buy papers",
             "i don't pay for second hand views of the illuminati","read","unknown","there are none","above",
             "not important","on holiday","none in print","d/n","its ok","none read","not read any","dont no","above",
             '"','11','12','42','n0','u','n/o','yes','n.a','n/k', 'm/a','na2', 'nk;', 'd.k',"n'a",
             "any news","none2","other","dont listen to radio","general radio","do not use","don't know any",
             "no websites","non biased","been on a cruise","don't mnkw","cant remember","did not watch",
             "don't watch tv","don't watch tv","i don't have a tv","i don't watch tv","tv","watch","can't recall",
             
            ]


crap_regex = ["^the "]
replace_dict = dict(zip(crap_list,[np.nan]*len(crap_list)))
to_strip = '.!? \n\t/,>-\\<`′\"\'$;'


In [790]:
# [x for x in df[match(df,"tv|radio|paper|internet",case_sensitive=False).index]\
#      .replace(replace_dict).replace(crap_regex,"",regex=True)\
#      .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
#      .replace(replace_dict).stack().value_counts().index if len(x)<=4]

In [ ]:
### Maybe it would be easier if I broke down the mappings by wave?

In [791]:
# df[match(df,"tv|radio|paper|internet",case_sensitive=False).index]\
#     .replace(replace_dict).replace(crap_regex,"",regex=True).stack().value_counts().head(400)[-60:]

                   

# pm? no? 1? 2? non? bt? 3? 4? 5?
# pm is a radio station!

# I think anything that's 1 letter is "woops - I want to get out of here" - except maybe "i" (what about 1/2/3 radio?)

In [806]:
tv_replace_dict = {"news night":"newsnight","questiontime":"question time","bbc 1":"bbc1","bbc 1 news":"bbc1 news","bbcnews":"bbc news",
    "bbc newsnight":"newsnight","party political broadcast":"party political broadcasts","bbc question time":"question time",
    "news at 10":"news at ten","bbc news at 10":"bbc news at ten","newnight":"newsnight",
    "leaders debates":"leaders debate","election debates":"election debate","debates":"debate",
    "daily politics show":"daily politics","andrew marr show":"andrew marr",
    "10 o clock news":"news at ten","10 oclock news":"news at ten","10pm news":"news at ten","ten o'clock news":"news at ten",
    "6 o clock news":"6 o'clock news","6 oclock news":"6 o'clock news","6pm news":"6 o'clock news",
    "six o clock news":"6 o'clock news","six o'clock news":"6 o'clock news",                 
    "news at 6":"6 o'clock news","news at six":"6 o'clock news",
    "aljazeera":"al jazeera",
    "andrew mar":"andrew marr","andrew mar show":"andrew marr","andrew marr on sunday":"andrew marr",
    "andrew marrs":"andrew marr","marr":"andrew marr","marr on sunday":"andrew marr","marr show":"andrew marr",
    "bbc  news":"bbc news","bbc new":"bbc news","bbc newa":"bbc news",
    "bbc 10pm news":"bbc news at ten",
    "bbc 6 o clock news":"bbc news at 6","bbc 6 oclock news":"bbc news at 6","bbc 6 o'clock news":"bbc news at 6",
    "bbc news 6pm":"bbc news at 6","bbc news at six":"bbc news at 6","bbc six o'clock news":"bbc news at 6",
    "bbc news24":"bbc news 24","bbc one":"bbc1","bbc one news":"bbc1 news",
    "big questions":"big question",
    "chanel 4 news":"channel 4 news","channel 4news":"channel 4 news","ch4 news":"channel 4 news",
    "ch 4 news":"channel 4 news","c4 news":"channel 4 news","channel four news":"channel 4 news","channel4 news":"channel 4 news",
    "chanel 4":"channel 4","channel4":"channel 4",
    "channel5":"channel 5",
    "daily politcs":"daily politics","daily politic":"daily politics","daily politic show":"daily politics",
    "daily politics bbc":"daily politics","daily polotics":"daily politics","daily poltics":"daily politics",
    "documentary":"documentaries",
    "euro news":"euronews",
    "good morning":"good morning britain",
    "hignfy":"have i got news for you",
    "it news ":"itn news","its news":"itn news",
    "itv  news": "itv news","itv new":"itv news","itvnews":"itv news",
    "itv news at ten":"itv news at 10",
    "leader debate":"leaders debate","leader debates":"leaders debate","leader's debate":"leaders debate",
    "leaders' debate":"leaders debate","leaders' debates":"leaders debate","leadership debate":"leaders debate",
    "leaders interviews":"leader interviews",
    "live debates":"live debate",
    "new":"news",
    "news channel":"news channels",
    "newsight":"newsnight","newsmight":"newsnight",
    "panarama":"panorama","panaroma":"panorama",
    "parliament channel":"parliament",
    "paxman interviews":"paxman",
    "peston":"peston on sunday",
    "pm question time":"pmqs","pm questions":"pmqs","pmq":"pmqs","pmq's":"pmqs",
    "prime minister's questions":"pmqs","prime ministers question time":"pmqs","prime ministers questions":"pmqs",
    "question  time":"question time","question times":"question time","question tine":"question time",
    "question tome":"question time","questions time":"question time","qustion time":"question time",
    "russia today":"rt",
    "scotland2015":"scotland 2015",
    "sky new":"sky news","skynews":"sky news",
    "today programme":"today",
    "tv debates":"debate",
    "election broadcast":"party political broadcasts","election broadcasts":"party political broadcasts",
    "party broadcast":"party political broadcasts","party broadcasts":"party political broadcasts",
    "party election broadcast":"party political broadcasts","party election broadcasts":"party political broadcasts",
    "political broadcast":"party political broadcasts","political broadcasts":"party political broadcasts",
    "political debate":"party political broadcasts","political debates":"party political broadcasts","ppb":"party political broadcasts",
    "party debate":"party debates","bbc news channel":"bbc news",
    "10 o'clock news":"news at ten","tv debate":"debate","live debate":"debate","news bbc":"bbc news",
    "parliament":"bbc parliament","7 leader debate":"7 leaders debate","7 party debate":"7 leaders debate",
    "7 way debate":"7 leaders debate","all party debate":"7 leaders debate","al jazera":"al jazeera",
    "andre marr":"andrew marr","andrew mars":"andrew marr","b.b.c. news":"bbc news","bbc 24 hour news":"bbc news 24",
    "bbc 24 hr news":"bbc news 24","bbc bews":"bbc news","bbc newd":"bbc news",
    "bbc parliament channel":"bbc parliament","bbc debates":"bbc debate","bbc election debate":"bbc debate",
    "bbc news night":"newsnight","bbc parliment":"bbc parliament","bbc politics show":"politics show",
    "bbc news 10pm":"bbc news at ten","bbc 6pm news":"bbc news at 6","bbc news at 6pm":"bbc news at 6",
    "bbc qt":"question time","bbc scottish news":"bbc scotland news","bbc tv":"bbc","bbc tv news":"bbc news",
    "bbc2 newsnight":"newsnight","bbcqt":"question time","bc news":"bbc news","bec news":"bbc news","broadcasts":"broadcast",
    "breakfast tv":"breakfast","c4":"channel 4","c4news":"channel 4 news","ch 4":"channel 4","ch4":"channel 4",
    "corrie":"coronation street","daily  politics":"daily politics","daily/sunday politics":"daily politics",
    "election specials":"election special","france24":"france 24","hardtalk":"hard talk","i tv news":"itv news",
    "htv":"itv","i t v":"itv","itv 1":"itv1","itv election debate":"itv debate","itv leader's debate":"itv leaders debate",
    "itv leaders' debate":"itv leaders debate","leaders interview":"leader interview","nesnight":"newsnight",
    "new night":"newsnight","newd":"news","news (bbc)":"bbc news","news itv":"itv news","news on bbc":"bbc news",
    "news on itv":"itv news","news broadcast":"news","news broadcasts":"news","news channels":"news",
    "news programme":"news","news programmes":"news","news programs":"news","other news":"news","newsnght":"newsnight",
    "opposition leaders debate":"opposition debate","parliament live":"parliament","party leaders debate":"leaders debate",
    "party political":"party political broadcasts","pm's questions":"pmqs","politic show":"politics show",
    "polotics show":"politics show","prime minister questions":"pmqs","q time":"question time","qestion time":"question time",
    "qt":"question time","question time bbc":"question time","queston time":"question time","quetion time":"question time",
    "scottish debate":"scottish leaders debate","sky newa":"sky news","sky news channel":"sky news","televised debates":"debate",
    "tv news":"news","scotland debate":"scotland debates","sky  news":"sky news","sky1":"sky","stv  news":"stv news",
    "sunday polotics":"sunday politics","sunday poltics":"sunday politics","scottish leaders' debate":"scottish leaders debate",
    "sunday politic":"sunday politics","bbc sunday politics":"sunday politics","six oclock news":"6 o'clock news",
    "bbc ten o'clock news":"bbc news at ten","10 o'clock news":"news at ten","10 pm news":"news at ten",
    "bbcnews24":"bbc news 24","news 24":"bbc news 24","news24":"bbc news 24","six oclock news":"6 o'clock news",
    "6 o'clock news bbc":"6 o'clock news","a week in politics":"week in politics","andrew marr sunday":"andrew marr",
    "bbc 2":"bbc2","bbc 24 news":"bbc news 24","bbc daily politics":"daily politics","bbc news 24":"bbc news 24","bbc news at one":"bbc news at 1",
    "bbc24":"bbc news 24","bbs news":"bbc news","calender":"calendar","chanel 5":"channel 5","channel  4 news":"channel 4 news",
    "daily poitics":"daily politics","daily politics bbc2":"daily politics","free speach":"free speech","general news":"news",
    "itv 3":"itv3","itv debates":"itv debate","itv live debate":"itv debate","leader interview":"leader interviews",
    "live debate":"debate","live leaders debate":"leaders debate","news channels":"news","news in general":"news",
    "news reports":"news bulletins","news sky":"sky news","newsnight bbc2":"newsnight","newsnoght":"newsnight",
    "northwest tonight":"north west tonight","party debate":"party debates","political programmes":"politics",                   
    "politics programme":"politics","question":"question time","charlie brooker's weekly wipe":"weekly wipe",
    "daily news":"news","4 news":"channel 4 news","5 news":"channel 5 news","bbc1news":"bbc1 news","bbc new 24":"bbc news 24",
    "bbc 2 newsnight":"newsnight","1st debate":"first debate","2nd debate":"second debate","any news programme":"news",
    "bbc regional news":"local bbc news","news channels":"news","bbc politics":"politics","it news":"itv news",
    "6 pm news":"6 o'clock news","al jazzera":"al jazeera","andrew  marr":"andrew marr","andrew marr programme":"andrew marr",
    "andrew neal":"andrew neil","andrew neill":"andrew neil","any other news":"news","b b c":"bbc","b b c news":"bbc news",
    "bbb news":"bbc news","bbbc news":"bbc news","bbc 24":"bbc news 24",
                   
                   
                   
                   
                   
}


vc = df[match(df,"tv",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(tv_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(120)[-60:]
# vc.head(300).sort_index().head(60)[-60:]
vc.head(60)[-60:]






# 1 o'clock news              5
# 24 hour news                6
# 2nd election debate         5
# 2nd leaders debate          4
# 9 o'clock news              5
# 9 oclock news               4

# it news -> prob itv rather than itn but ambiguous!
# leaders -> ambiguous (debate/interview)
# hh,i,

96086 0.8768082759194888 0.9220281830859854


bbc news                      17494
question time                 10648
newsnight                      9613
news                           8838
sky news                       4883
itv news                       3978
daily politics                 3909
channel 4 news                 2785
andrew marr                    2522
sunday politics                1594
this week                      1544
bbc                            1432
debate                         1155
itv                            1011
leaders debate                 1007
panorama                        740
bbc1                            716
news at ten                     675
bbc news 24                     649
bbc breakfast                   560
politics show                   503
party political broadcasts      489
scotland tonight                450
local news                      439
sky                             435
channel 4                       349
stv news                        317
election debate             

In [769]:
radio_replace_dict = {"radio 5 live":"5 live","today programme":"today","bbc radio 4":"radio 4","bbc radio 2":"radio 2",
    "five live":"5 live","radio4":"radio 4","radio2":"radio 2","bbc2":"radio 2","radio2":"radio 2","bbc4":"radio 4",
    "5live":"5 live","bbc 2":"radio 2","bbc 4":"radio 4","bbc1":"radio 1","bbc radio 1":"radio 1","bbc 5 live":"5 live",
    "bbc radio":"bbc","bbc world service":"world service","today program":"today","radio news":"news","radio1":"radio 1",
    "r4":"radio 4","radio 4 today":"today","today radio 4":"today","today on radio 4":"today","r4 today":"today","today r4":"today",
    "world at 1":"world at one","bbc radio 2 news":"radio 2 news","bbc radio 4 news":"radio 4 news","radio five live":"5 live",
    "bbc five live":"5 live","radio one":"radio 1","radio 4 today programme":"today","radio four":"radio 4","bbc today":"today",
    "bbc radio 5 live":"5 live","bbc today":"today","r4 news":"radio 4 news","today show":"today","bbc radio 5":"radio 5",
    "pienaars politics":"pienaar's politics","five live drive":"5 live drive","radio two":"radio 2","radio5":"radio 5",
    "6 o'clock news":"six o'clock news","bbc 1":"radio 1","r2":"radio 2","radio five":"radio 5","today (radio 4)":"today",
    "today prog":"today","bbc r2":"radio 2","bbc r4":"radio 4","r4 today programme":"today","6pm news":"six o'clock news",
    "today, radio 4":"today","bbc radio2":"radio 2","radio 5live":"5 live","today (r4)":"today","today - radio 4":"today",
    "bbc r4 today":"today","radio3":"radio 3","today programme radio 4":"today","today on r4":"today","bbc2 news":"radio 2 news",
    "bbc radio two":"radio 2","bbc 5":"radio 5","new":"news","news radio 4":"radio 4 news","fivelive":"5 live",
    "bbc 2 news":"radio 2 news","radio4 news":"radio 4 news","bbc 5live":"5 live","radio4 today":"today",
    "today on radio4":"today","radio5 live":"5 live","bbc radio four":"radio 4","bbc radio 1 news":"radio 1 news",
    "radio four news":"radio 4 news","today radio4":"today","pienars politics":"pienaar's politics",
    "pinaars politics":"pienaar's politics","r5 live":"5 live","radio2 news":"radio 2 news","bbc4 news":"radio 4 news",
    "bbc4 today":"today","bbc5":"radio 5","bbc radio 3":"radio 3","bbc r4 news":"radio 4 news","local":"local radio",
    "bbc scotland":"radio scotland","radio scotland news":"radio scotland","bbc local radio":"local radio",
    "local radio news":"local news","radio 4 pm":"pm","jeremy vine show":"jeremy vine","jeremy vine radio 2":"jeremy vine",
    "bbc radio news":"bbc news","pm radio 4":"pm","talk sport":"talksport","bbc wales":"radio wales",
    "five live breakfast":"5 live breakfast","pm on radio 4":"pm","drive":"5 live drive",
    "heart fm":"heart","capital fm":"capital","p.m":"pm","women's hour":"woman's hour",
    "5live breakfast":"5 live breakfast","radio 2 jeremy vine":"jeremy vine","bbc radio 4 today":"today",
    "pm programme":"pm","radio 1 newsbeat":"newsbeat","r4 pm":"pm","lbc nick ferrari":"nick ferrari",
    "womens hour":"woman's hour","pm r4":"pm","news on radio 4":"radio 4 news","five live news":"5 live news",
    "smooth radio":"smooth","absolute radio":"absolute","bbc radio wales":"radio wales","local bbc":"local radio","bbc local":"local radio",
    "bbc 4 news":"radio 4 news","womans hour":"woman's hour","bbc3":"radio 3","vine":"jeremy vine","bbc radio one":"radio 1",
    "bbc radio4":"radio 4","r5":"radio 5","nick ferrari lbc":"nick ferrari","local bbc radio":"local radio",
    "5live drive":"5 live drive","bbc pm":"pm","bbc newsbeat":"newsbeat","classic":"classic fm","local station":"local radio",
    "jeremy vine r2":"jeremy vine","lbc radio":"lbc","lbc 97.3":"lbc","questiontime":"question time","bbc r4 pm":"pm",
    "local radio station":"local radio","j vine":"jeremy vine","bbc local news":"local news",
    "jeremy vine on radio 2":"jeremy vine","pm - radio 4":"pm","r1":"radio 1","radio 4 any questions":"any questions",
    "pm (radio 4)":"pm","5live news":"5 live news","bbc radio 4 pm":"pm","jermey vine":"jeremy vine",
    "r4 world at one":"world at one","bbc radio solent":"radio solent","news on radio 2":"radio 2 news",
    "local bbc news":"local news","bbc radio 2 jeremy vine":"jeremy vine","bbc radio 4 today programme":"today",
    "ian dale":"lbc iain dale","classical fm":"classic fm","bbc world service news":"world service news",
    "pm at 5pm":"pm","nick ferrari on lbc":"nick ferrari","pinnars politics":"pienaar's politics","p m":"pm",
    "news broadcasts":"news bulletins","pinaar's politics":"pienaar's politics","bbc radio4 news":"radio 4 news",
    "r3":"radio 3","bbc 3":"radio 3","capital radio":"capital","piennars politics":"pienaar's politics",
    "bbc radio1":"radio 1","bbc one":"radio 1","news bbc":"bbc news","bbc radio 5 news":"radio 5 news",
    "radio  4 in general":"radio 4","radio 4 programmes":"radio 4","radio 4 generally":"radio 4",
    "jeremey vine":"jeremy vine","bbc radio 1 newsbeat":"newsbeat","r2 news":"radio 2 news","pm (r4)":"pm",
    "news radio 2":"radio 2 news","jeremy vine bbc radio 2":"jeremy vine","pienar's politics":"pienaar's politics",
    "smooth fm":"smooth","james o'brien lbc":"james o'brien","iain dale lbc":"lbc iain dale","bbc radio five live":"5 live",
    "today programme r4":"today","today programe":"today","pinar's politics":"pienaar's politics",
    "pienar's politics":"pienaar's politics","any question":"any questions","pm, radio 4":"pm",
    "radio5live":"radio 5 live","jeremy vine (radio 2)":"jeremy vine","radio 4 today program":"today",
    "radio 4 today prog":"today","local bbc station":"local radio","deadringers":"dead ringers","radio two news":"radio 2 news",
    "radio 5 live":"5 live","kiss fm":"kiss","news reports":"news bulletins","bbc news bulletins":"news bulletins",
    "radio one news":"radio 1 news","qt":"question time","jeramy vine":"jeremy vine","radio 2 jeremy vine show":"jeremy vine",
    "bb2":"radio 2","ian dale lbc":"lbc iain dale","6 o clock news":"six o'clock news","6 oclock news":"six o'clock news",
    "6 pm news":"six o'clock news","bbc radio scotland":"radio scotland","bbc scotland news":"radio scotland news",
    "bbc radio scotland news":"radio scotland news","chris evans breakfast show":"chris evans","clyde1":"clyde 1",
    "debates":"debate","drivetime":"drive time","file on 4":"file on four","forth 1":"forth one","heart fm news":"heart news",
    "heart radio":"heart","jeremyvine":"jeremy vine","money box":"moneybox","money programme":"moneybox",
    "news bulletin":"news bulletins","news programmes":"news bulletins","other news":"news","pm at 5 pm":"pm",
    "pmq":"pmqs","pmq's":"pmqs","breakfast 5 live":"5 live breakfast","five live daily":"5 live daily",
    "general news":"news","iain dale (lbc)":"lbc iain dale","john humphreys":"john humphries","newa":"news",
    "news bullitins":"news bulletins","6music news":"6 music news","nick ferrari (lbc)":"nick ferrari",
    "question time extra":"question time extra time","radio 4 in general":"radio 4","radio 4 news bulletins":"radio 4 news",
    "radio 4 news at 6":"radio 4 6pm news","radio 4 6 o'clock news":"radio 4 6pm news","radio 4 six o'clock news":"radio 4 6pm news",
    "real radio":"real","six o clock news":"six o'clock news","tfm":"tay fm","today at westminster":"today in westminster",
    "today in parliment":"today in parliament","various radio 4":"radio 4","week at westminster":"week in westminster",
    "westminister hour":"westminster hour","westminster today":"today in westminster","world at one r4":"world at one",
    "today bbc 4":"today","today r 4":"today","you & yours":"you and yours",
    "news at 1":"news at one","one o'clock news":"news at one","1 o'clock news":"news at one","1pm news":"news at one",
    "news at 6":"six o'clock news","news at six":"six o'clock news","ten o'clock news":"news at 10","news at ten":"news at 10",
    "10pm news":"news at 10","10 o'clock news":"news at 10","bbc world news":"world news",
                      
}





vc = df[match(df,"radio",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(radio_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(300).sort_index().head(300)[-60:]
vc.head(170)[-60:]


# radio 1-5 (news)
# bbc (news|london)
# local (radio|news)
# classic fm (news)|heart (news)
# news|news bulletins
# pm|lbc|today|jeremy vine|news quiz|world service|capital|smooth|talksport|pienaar's politics|newsbeat

# 5 live (drive|breakfast|news)
# world at one|any questions|any answers|question time|westminster hour|today in parliament|week in westminster
# radio (scotland|wales)
# good morning (scotland|wales)|world tonight|broadcasting house|woman's hour|magic|kiss|wato|gms


# radio 4 morning/breakfast/morning news???
            
# 5 o'clock news               9
# 6 o'clock news radio 4       5
#                   4

# am, fm -> useful??
# free radio,hourly news,world news,national news
# radio clyde


38123 0.7992550428874957 0.863048553366734


pmqs                          15
scottish news                 14
bbc world news                14
this morning                  14
politics                      14
stephen nolan                 14
real                          14
chris evans                   14
radio devon                   13
metro                         13
radio wales news              13
gold                          13
file on four                  13
politics today                13
radio 4 am                    13
radio kent                    12
good morning                  12
radio wm                      12
daily politics                12
morning call                  12
radio 4 morning news          12
scotland today                12
radio sheffield               12
nicky campbell                11
radio newcastle               11
morning news                  11
wave                          11
taro'r post                   11
radio humberside              11
6 music news                  11
midnight n

In [770]:
paper_replace_dict = {"mail":"daily mail","daily telegraph":"telegraph","daily mirror":"mirror","daily express":"express",
    "independant":"independent","financial times":"ft","sunday mail":"mail on sunday","standard":"evening standard",
    "local newspaper":"local paper","record":"daily record","daily star":"star","gaurdian":"guardian","dailymail":"daily mail",
    "press & journal":"press and journal","i paper":"i","d telegraph":"telegraph","dail mail":"daily mail",
    "gardian":"guardian","mirrow":"mirror","gurdian":"guardian","telegragh":"telegraph","mos":"mail on sunday",
    "indipendent":"independent","ipaper":"i","daly mail":"daily mail","huffpost":"huffington post",
    "huffington":"huffington post","local press":"local paper","p&j":"press and journal","i newspaper":"i",
    "mail on line":"mail online","d mail":"daily mail","huff post":"huffington post","indy":"independent",
    "saturday mail":"mail on saturday","grauniad":"guardian","daily  mail":"daily mail","guradian":"guardian",
    "dt":"telegraph","telegrapgh":"telegraph","london standard":"evening standard","torygraph":"telegraph",
    "indie":"independent","dailymirror":"mirror","mailonline":"mail online","guadian":"guardian","local news":"local paper",
    "thesun":"sun","local daily":"local paper","es":"evening standard","guardain":"guardian",
    "local papers":"local paper","indpendent":"independent","sunday sun":"sun on sunday","men":"manchester evening news",
    "s times":"sunday times","bbc website":"bbc online","express & star":"express and star","local newspapers":"local paper",
    "indi":"independent","guardian weekly":"guardian","bbc news online":"bbc online","exspress":"express","sun times":"sunday times",
    "daily  mirror":"mirror","local news paper":"local paper","daily maik":"daily mail","telagraph":"telegraph",
    "d.mail":"daily mail","local weekly paper":"local paper","d. telegraph":"telegraph","dm":"daily mail",
    "indepedent":"independent","telegrah":"telegraph","local rag":"local paper","d mirror":"mirror","indipendant":"independent",
    "huffington post uk":"huffington post","daily mirrow":"mirror","mirroe":"mirror","daily telegrapgh":"telegraph",
    "indepentant":"independent","daily mail online":"mail online","telegrath":"telegraph","local daily paper":"local paper",
    "telegaph":"telegraph","tines":"times","observor":"observer","huffingdon post":"huffington post",
    "guaridan":"guardian","dailey express":"express","timed":"times","the sun":"sun","local weekly":"local paper",
    "e standard":"evening standard","indepenent":"independent","independence":"independent","dailly mail":"daily mail",
    "guardiam":"guardian","sundaytimes":"sundaytimes","daily miror":"mirror","theguardian":"guardian","gusrdian":"guardian",
    "daliy mail":"daily mail","daiy mail":"daily mail","sunday time":"sunday times","mirrror":"mirror","daiky mail":"daily mail",
    "huffing ton post":"huffington post","idependent":"independent","guardin":"guardian",
    "daily mail on line":"mail online","daily morror":"mirror","times on sunday":"sunday times","f t":"ft",
    "ns":"new statesman","sum":"sun","son":"sun","daiy telegraph":"telegraph","miror":"mirror","dailytelegraph":"telegraph",
    "p & j":"press and journal","daily times":"times","f.t":"ft","mirrir":"mirror","dail mirror":"mirror",
    "daily fail":"daily mail","telgraph":"telegraph","local guardian":"guardian","daily mai":"daily mail",
    "regional paper":"local paper","dailystar":"star","bbc news website":"bbc news online","daliy mirror":"mirror",
    "m.e.n":"manchester evening news","inderpendant":"independent","p and j":"press and journal",
    "d. mail":"daily mail","daily telegragh":"telegraph","inependent":"independent","giuardian":"guardian","morrir":"mirror",
    "telegrpah":"telegraph","huffpo":"huffington post","d tel":"telegraph","sat times":"saturday times","i daily":"i",
    "evening stardard":"evening standard","daily mirrpr":"mirror","daily rxpress":"express","telegraphy":"telegraph",
    "daily telrgraph":"telegraph","daily maul":"daily mail","local daily newspaper":"local paper","tmes":"times",
    "telegrap[h":"telegraph","daily teleghraph":"telegraph","daily  express":"express","guardians":"guardian",
    "evening std":"evening standard","evening stamdard":"evening standard","local area paper":"local paper",
    "dailyexpress":"daily express","telegrph":"telegraph","mrror":"mirror","daily mail;":"daily mail",
    "dailt mail":"daily mail","aun":"sun","saturday daily mail":"mail on saturday","thetimes":"times","un":"sun",
    "daily mail on sunday":"mail on sunday","the i":"i","indepentdent":"independent","ii":"i","telegrap":"telegraph",
    "i dependant":"independent","huffinton post":"huffington post","guarduian":"guardian","indepenant":"independent",
    "dail;y mail":"daily mail","timea":"times","telrgraph":"telegraph","evening  standard":"evening standard",
    "dsily mail":"daily mail","dail telegraph":"telegraph","guardia":"guardian","daily mirrior":"mirror",
    "daily  telegraph":"telegraph","press& journal":"press and journal","oberver":"observer","dailey mirror":"mirror",
    "telegrapth":"telegraph","localpaper":"local paper","mirro":"mirror","daily mial":"daily mail","insependent":"independent",
    "sunday mirrow":"sunday mirror","new stateman":"new statesman","daily express":"express","daily t":"telegraph",
    "telepgraph":"telegraph","f times":"ft","imes":"times","aberdeen press & journal":"press and journal",
    "aberdeen press and journal":"press and journal","courier and advertiser":"courier","couriet":"courier",
    "dailt telegraph":"telegraph","daily express":"express","daily maily":"daily mail","daily recored":"daily record",
    "eastern daily press":"edp","edp24":"edp","eveningstandard":"evening standard","guardian.co.uk":"guardian online",
    "guardion":"guardian","glasgow hearld":"herald","glasgow herald":"herald","hearld":"herald","herald (glasgow)":"herald",
    "herald/sunday herald":"herald","herlad":"herald","herold":"herald","i/independent":"independent/i",
    "leics mercury":"leicester mercury","lep":"lancashire evening post","mail on sunda":"mail on sunday",
    "mail/mail on sunday":"daily mail","mirtor":"mirror","nation":"national","nationa":"national",
    "national (scotland)":"national","ny times":"new york times","press":"press and journal","s telegraph":"telegraph",
    "sat telegraph":"saturday telegraph","s. times":"times","scotland in sunday":"scotland on sunday",
    "yep":"yorkshire evening post","tlegraph":"telegraph",
    "daily  record":"daily record","daily mirroe":"mirror","dmail":"daily mail","e.d.p":"edp",
    "eadt":"east anglian daily times","east anglian":"east anglian daily times","fh":"falkirk herald",
    "fife press":"fife free press","independent.co.uk":"independent online","local":"local paper",
    "local evening paper":"local paper","local evening post":"local paper","local free paper":"local paper",
    "local gazette":"local paper","local to my area":"local paper","mai":"daily mail","mail  on sunday":"mail on sunday",
    "mi rror":"mirror","msil":"daily mail","new shopper":"news shopper","obseerver":"observer","sun  y":"sun",
    "scotland on sundat":"scotland on sunday","sundaytimes":"sunday times","western morning ~news":"western morning news",
    "(glasgow) herald":"herald","bournemouth daily echo":"bournemouth echo","daily mail/mail on sunday":"daily mail",

                      
                      
}
vc = df[match(df,"paper",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(paper_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(300).sort_index().head(60)[-60:]
vc.head(180)[-60:]
# .head(300)[-60:]

# ny times                    5
# 
# mostly blogs???

# guardian/the observer          2
# broadsheets


55388 0.9560915721817 0.9759334151801834


google news                5
buzzfeed                   5
argus                      5
camden new journal         5
sport                      5
ipswich star               5
york press                 5
sunderland echo            5
sunday independent         5
nationalist                5
independent/i              5
derby telegraph            5
socialist                  5
times on saturday          4
ross-shire journal         4
golwg                      4
scottish express           4
worcester news             4
local echo                 4
bournemouth echo           4
independent online         4
daily echo                 4
bolton news                4
evening telegraph          4
evening gazette            4
post                       4
southern reporter          4
dandy                      4
today                      4
fife free press            4
lichfield mercury          4
herald express             4
falkirk herald             4
times online               3
oban times    

In [771]:
internet_replace_dict = {"bbc.co.uk":"bbc","mail":"daily mail","bbc website":"bbc","mail online":"daily mail",
    "daily telegraph":"telegraph","bbc news website":"bbc news","mail on line":"daily mail","you gov":"yougov",
    "daily mirror":"mirror","guardian.co.uk":"guardian","bbc online":"bbc","huff post":"huffington post",
    "bbc.co.uk/news":"bbc news","mailonline":"daily mail","huffington":"huffington post","daily express":"express",
    "bbc news online":"bbc news","telegraph.co.uk":"telegraph","bbcnews":"bbc news","guardian website":"guardian",
    "bbc news app":"bbc news","dailymail":"daily mail","huffpost":"huffington post","huffing ton post":"huffington post",
    "financial times":"ft","daily mail online":"daily mail","telegraph online":"telegraph","www.bbc.co.uk":"bbc",
    "guardian.com":"guardian","labourlist":"labour list","gaurdian":"guardian","ft.com":"ft",
    "dailymail.co.uk":"daily mail","news.bbc.co.uk":"bbc news","bbc.com":"bbc","theguardian.com":"guardian",
    "russia today":"rt","face book":"facebook","rt.com":"rt","politicalbetting.com":"politicalbetting",
    "bellacaledonia":"bella caledonia","independent.co.uk":"independent","may2015.com":"may2015",
    "independent online":"independent","sky.com":"sky","you tube":"youtube","bbc news site":"bbc news",
    "bec news":"bbc news","telegraph website":"telegraph","bbcnews.co.uk":"bbc news","independent website":"independent",
    "huffington post uk":"huffington post","daily mail on line":"daily mail","daily mail website":"daily mail",
    "local newspaper":"local paper","bbc/news":"bbc news","msm":"msn","www.dailymail.co.uk":"daily mail","msn.com":"msn",
    "thetimes.co.uk":"times","www.telegraph.co.uk":"telegraph","bbc new":"bbc news","timesonline":"times",
    "huffingdon post":"huffington post","sky news app":"sky news","huffinton post":"huffington post",
    "mirror.co.uk":"mirror","coffee house":"spectator coffee house","www.guardian.co.uk":"guardian","bbc app":"bbc",
    "guardian news":"guardian","newstatesman":"new statesman","gardian":"guardian","theguardian.co.uk":"guardian",
    "goggle":"google","itv.com":"itv","facebook links":"facebook","telegraph on line":"telegraph",
    "bella caladonia":"bella caledonia","bbc homepage":"bbc","dailymail.com":"daily mail","huff":"huffington post",
    "dt":"telegraph","mirror on line":"mirror","indy":"independent","ft online":"ft","newstatesman.com":"new statesman",
    "guardian on line":"guardian","mirror online":"mirror","times on line":"times","utube":"youtube",
    "daly mail":"daily mail","u gov":"yougov","bbc web page":"bbc","huffingtonpost.co.uk":"huffington post",
    "guardian app":"guardian","times website":"times","indipendant":"independent","bbc.co,uk":"bbc",
    "express.co.uk":"express","bbc news on line":"bbc news","sun.co.uk":"sun","bbc bews":"bbc news","telegrapth":"telegraph",
    "gurdian":"guardian","bbcnews.com":"bbc news","bbc news channel":"bbc news","guadian":"guardian",
    "bbc home":"bbc","aljazeera":"al jazeera","bbc 1":"bbc1","bbc news.co.uk":"bbc news","bbc on line":"bbc",
    "bbc web site":"bbc","bt home page":"bt","bt internet":"bt","bt.com":"bt","buzz feed":"buzzfeed","commonspace":"common space",
    "conhome":"conservative home","conservative":"conservative home","conservativehome":"conservative home",
    "conservativehome.com":"conservative home","dail mail":"daily mail","email":"emails","fb":"facebook",
    "googlenews":"google news","flip board":"flipboard","golwg360":"golwg 360","government":"gov.uk",
    "green":"green party","green party website":"green party","guardian":"guardian","guido":"guido fawkes",
    "guido fawkes blog":"guido fawkes","herald online":"herald","huffingtonpost":"huffington post",
    "huffpo":"huffington post","independant":"independent","independent newspaper":"independent",
    "liberal democrat voice":"lib dem voice","libdemvoice":"lib dem voice","lord ashcroft polls":"lord ashcroft",
    "may 2015":"may2015","moneysavingexpert":"money saving expert","mse":"money saving expert","msn uk":"msn",
    "msn.co.uk":"msn","new statesman website":"new statesman","newsnet.scot":"newsnet scotland",
    "newsnetscotland":"newsnet scotland","news net scotland":"newsnet scotland","news sites":"news","news websites":"news",
    "ny times":"new york times","order-order":"order order","order-order.com":"order order",
    "political betting.com":"political betting","politicalbetting":"political betting","politics uk":"politics.co.uk",
    "politicshome":"politics home","scotland goes pop":"scot goes pop","scotsman.com":"scotsman","sk":"sky",
    "sky new":"sky news","sky news online":"sky news","sky news website":"sky news","skynews":"sky news",
    "redbox":"red box","reddit.com":"reddit","skynews.co.uk":"sky news","skynews.com":"sky news","snp website":"snp",
    "snp.org":"snp","stv.tv":"stv","sun online":"sun","talk talk":"talktalk","telegrah":"telegraph","telegraph.com":"telegraph",
    "times online":"times","times.co.uk":"times","twitter.com":"twitter","uk polling":"uk polling report",
    "ukip website":"ukip","ukpolling report":"uk polling report","ukpollingreport":"uk polling report",
    "polling report":"uk polling report","virgin media":"virgin","virginmedia":"virgin","wales on line":"wales online",
    "walesonline":"wales online","weegingerdug":"wee ginger dug","wings":"wings over scotland",
    "wingsoverscotland":"wings over scotland","wingsoverscotland.com":"wings over scotland","www.bbc.co.uk/news":"bbc news",
    "yahoo home page":"yahoo","yahoo.co.uk":"yahoo","yahoo.com":"yahoo","yougov.co.uk":"yougov","yougov.com":"yougov",
    "zero hedge":"zerohedge","wos":"wings over scotland","ukpr":"uk polling report","38degrees":"38 degrees",
    "aol.com":"aol","aav":"another angry voice","bbc  news":"bbc news","bbc i player":"bbc iplayer","bbc newd":"bbc news",
    "bbc news 24":"bbc news","bbc news web site":"bbc news","bbc online news":"bbc news","bbc uk":"bbc","bbc,co.uk":"bbc",
    "bbd news":"bbc news","bbs news":"bbc news","bells caledonia":"bella caledonia","brietbart":"breitbart",
    "bt homepage":"bt","btyahoo":"bt yahoo","c4":"channel 4","channel4":"channel 4","c4 news":"channel 4 news",
    "conservative party website":"conservative party","conservatives":"conservative party","conservative website":"conservative party",
    "digitalspy":"digital spy","economist.com":"economist","election forecast":"electionforecast.co.uk",
    "electoralcalculus.co.uk":"electoral calculus","eureferendum":"eu referendum","facebook groups":"facebook",
    "facebook.com":"facebook","france24":"france 24","gogle":"google","google search":"google","google.com":"google",
    "greens":"green party","guidofawkes":"guido fawkes","herald website":"herald","heraldscotland.com":"herald scotland",
    "independant.co.uk":"independent","itv.co.uk":"itv","labour party website":"labour party","labour website":"labour party",
    "libdem voice":"lib dem voice","mail plus":"daily mail","mail.co.uk":"daily mail","news feeds":"news",
    "political  betting":"political betting","politicalbetting":"political betting","reuter":"reuters","reuters.co.uk":"reuters",
    "sky.co.uk":"sky","telegragh":"telegraph","telegraph app":"telegraph","telegraph blogs":"telegraph","thesun":"sun",
    "thisismoney":"this is money","uk gov":"gov.uk","whatscotlandthinks":"what scotland thinks","www.snp.org":"snp",
    "spectator coffeehouse":"spectator coffee house","www.bbcnews":"bbc news","www.ukpollingreport.co.uk":"uk polling report",
    "www.yougov.com":"yougov","yarhoo":"yahoo","you guv":"yougov","youyube":"youtube","guardian online":"guardian",
                         
                         
                         
                         
                         
}
vc = df[match(df,"internet",case_sensitive=False).index].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(internet_replace_dict).replace("",np.nan).stack().value_counts()
print( vc.sum(),vc.head(60).sum()/vc.sum(),vc.head(300).sum()/vc.sum() )
# vc.head(300).sort_index().head(300)[-60:]
vc.head(170)[-60:]
# .head(300)[-60:]



# party website                  6
# party websites                32
# political party websites       4
# individual party websites       5  
# party website                6
# party websites              32
# political party websites       4

# daily telegraph website
#five thirty eight             4 
# general                         3
# general browsing                3
# local                        9
# local news                   9
# local paper                 28
# newspapers                   9
# various blogs

# other- alternative,rather not say

50715 0.874573597554964 0.9252883762200532


virgin                     17
spiked                     16
vox political              16
france 24                  15
emails                     15
sunday herald              15
golwg 360                  14
left foot forward          14
bbc scotland               14
itn                        14
digital spy                14
common weal                14
bbc iplayer                13
canary                     13
mumsnet                    13
bt yahoo                   13
euronews                   13
washington post            12
full fact                  12
glasgow herald             12
lord ashcroft              12
capx                       12
week                       11
zerohedge                  11
orange                     11
daily wales                11
commentator                11
electionforecast.co.uk     10
daily post                 10
le monde                   10
lbc                        10
daily mash                 10
eu referendum              10
news now  

In [695]:
# df[match(df,"tv|radio|paper|internet",case_sensitive=False).index].replace(replace_dict)\
#     .replace(crap_regex,"",regex=True)\
#     .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
#     .replace(replace_dict).stack().value_counts().head(60)[-60:]




In [854]:
%%time
tvradiopaperinternet = pd.DataFrame()

tv_cols = match(df,"tv",case_sensitive=False).index
tvradiopaperinternet[tv_cols] = df[tv_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(tv_replace_dict).replace("",np.nan).astype('category')

radio_cols = match(df,"radio",case_sensitive=False).index
tvradiopaperinternet[radio_cols] = df[radio_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(radio_replace_dict).replace("",np.nan).astype('category')

paper_cols = match(df,"paper",case_sensitive=False).index
tvradiopaperinternet[paper_cols] = df[paper_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(paper_replace_dict).replace("",np.nan).astype('category')

internet_cols = match(df,"internet",case_sensitive=False).index
tvradiopaperinternet[internet_cols] = df[internet_cols].replace(replace_dict)\
    .replace(crap_regex,"",regex=True)\
    .applymap(lambda x: x.strip(to_strip) if pd.notnull(x) else np.nan)\
    .replace(replace_dict).replace(internet_replace_dict).replace("",np.nan).astype('category')

Wall time: 1min 49s


In [858]:
tvradiopaperinternet[tvradiopaperinternet.columns[0]].value_counts()

bbc news                     5863
news                         3429
question time                2358
newsnight                    2120
daily politics               1052
                             ... 
news (to track the bias)        1
news (sky)                      1
news (rt or al jazeera)         1
news (itn)                      1
(only foreign programmes)       1
Name: tv1_1W4, Length: 1010, dtype: int64

In [859]:
tvradiopaperinternet.to_pickle('../BES_analysis_data/W19_comb_strings\\tvradiopaperinternet.zip', compression='zip')

In [ ]:
#####
### q1 variable section
# Terrestrial programmes watched before yesterday/yesterday/today

In [861]:
ser = pd.Series( flatten( df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan).stack().values ) )
ser.value_counts()

#.value_counts().head(60)[-60:]

                                                                                   385135
201911172200|bbc1|bbc news|news                                                       589
201911242200|bbc1|bbc news|news                                                       572
201911192000|itv1|johnson v corbyn: the itv debate|documentary: current affairs       567
201912012200|bbc1|bbc news|news                                                       555
                                                                                    ...  
20191202210                                                                             1
201911142245|bbc1|question time|current a                                               1
201911142245|bbc1|question time|current                                                 1
201911221900|bbc1|question time leaders special|current affai                           1
201912062230|itv1|itv new                                                               1
Length: 33

In [862]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan).stack().value_counts().head(180)[-60:]

201911161830|channel 4|channel 4 news|news                                                                                                                    53
201912071720|bbc1|regional news and weather|news                                                                                                              52
201911171830|channel 4|channel 4 news|news                                                                                                                    52
201912090600|bbc1|breakfast|daytime                                                                                                                           51
201911231800|itv1|itv evening news|news~201911231815|itv1|regional news and weather|news                                                                      51
201911242230|itv1|itv news|news                                                                                                                               50
201911261800|bbc1|bbc news at six|

In [863]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).stack().value_counts()

1    410563
0    144759
2     13861
3      6398
4      3367
5      2155
6      1508
7       336
8         1
dtype: int64

In [871]:
(df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0)==8).sum()

q1_before_yesterdayW18         0
q1_before_yesterday_newsW18    0
q1_yesterdayW18                0
q1_yesterday_newsW18           1
q1_todayW18                    0
q1_today_newsW18               0
dtype: int64

In [875]:
df["q1_yesterday_newsW18"][df["q1_yesterday_newsW18"].replace("__na__",np.nan)\
           .apply(lambda x: len(x.split("~")) if pd.notnull(x) else 0)==8]

id
78922    201912011000|skynews|sky news|news~20191201120...
Name: q1_yesterday_newsW18, dtype: category
Categories (2631, object): [, 201911130600|bbcnews24|breakfast|news, 201911130600|bbcnews24|breakfast|news~20191113..., 201911130600|bbcnews24|breakfast|news~20191113..., ..., 20191210900|skynews|all out politics|news~2019..., 20191210900|skynews|all out politics|news~2019..., 20191211700|skynews|kay burley @breakfast|news, __na__]

In [877]:
df.loc[78922,"q1_yesterday_newsW18"].split("~")
# okay - we'll never actually *need* all 8 columns

['201912011000|skynews|sky news|news',
 '201912011200|bbcnews24|bbc news|news',
 '201912011230|bbcnews24|click|news',
 '201912011300|bbcnews24|bbc news|news',
 '201912011100|skynews|sky news|news',
 '201912011400|bbcnews24|bbc news|news',
 '201912011200|skynews|sky news|news',
 '20191']

In [164]:
# ser.value_counts().apply(lambda x: len(x.index.split("|")))
section_no = [len(x.split("|")) for x in ser.value_counts().index]
sec_no = pd.Series(section_no)
sec_no.value_counts()

4    2353
3     780
2     142
1     109
dtype: int64

In [222]:
ser.value_counts().loc[ser.value_counts().index[sec_no>=2]].sort_index().head(60)

201911111800|itv1|regional news and weather|news                                          1
201911112200|bbc1|bbc news at ten|news                                                    1
201911120600|bbc1|breakfast|daytime                                                     175
201911120600|bbcnews24|breakfast|news                                                    14
201911120600|itv1|good morning britain|daytime                                           47
201911120600|skynews|the early rundown|news                                               6
201911120700|skynews|kay burley @breakfast|news                                          34
201911120830|bbcnews24|worklife|news                                                      3
201911120830|itv1|lorraine|daytime                                                        8
201911120900|bbc2|bbc news at nine|daytime                                               13
201911120900|bbcnews24|bbc news at nine|news                                    

In [227]:
ser.apply(lambda x: "|".join(x.split("|")[1:]))[ser.apply(lambda x:len(x.split("|")))>=2].value_counts().sort_index().head(60)

                                                     23
b                                                    13
bb                                                   16
bbc                                                  12
bbc1                                                 12
bbc1|                                                 7
bbc1|ask the leader                                   1
bbc1|ask the leaders|current af                       1
bbc1|ask the leaders|current affairs:                 1
bbc1|ask the leaders|current affairs: politics       59
bbc1|b                                                3
bbc1|bb                                               3
bbc1|bbc                                              3
bbc1|bbc london|news                                 34
bbc1|bbc n                                            5
bbc1|bbc ne                                           5
bbc1|bbc new                                          5
bbc1|bbc news                                   

In [305]:
year_month_day_24hr = np.unique([x for x in ser.apply(lambda x: x.split("|")[0])[ser.apply(lambda x:len(x.split("|")))>=2].value_counts().sort_index().index])

In [308]:
np.unique([x[0:4] for x in year_month_day_24hr])
# all 2019

array(['2019'], dtype='<U4')

In [309]:
np.unique([x[4:6] for x in year_month_day_24hr])
# 11/12

array(['11', '12'], dtype='<U2')

In [310]:
np.unique([x[6:8] for x in year_month_day_24hr])

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30'], dtype='<U2')

In [311]:
np.unique([x[8:12] for x in year_month_day_24hr])

array(['0600', '0700', '0830', '0900', '0915', '0930', '1000', '1030',
       '1100', '1110', '1130', '1200', '1210', '1230', '1300', '1330',
       '1355', '1400', '1430', '1500', '1530', '1600', '1630', '1645',
       '1700', '1705', '1710', '1715', '1720', '1725', '1730', '1735',
       '1745', '1750', '1755', '1800', '1805', '1810', '1815', '1820',
       '1825', '1830', '1835', '1840', '1845', '1850', '1855', '1900',
       '1910', '1915', '1925', '1930', '1945', '1955', '2000', '2005',
       '2030', '2045', '2100', '2105', '2110', '2130', '2200', '2205',
       '2210', '2215', '2220', '2225', '2230', '2235', '2245', '2250',
       '2255', '2300', '2315', '2330', '2345', '600', '700', '830', '900',
       '915', '930'], dtype='<U4')

In [313]:
np.unique([x[0:8] for x in year_month_day_24hr])
# 11/11 - 12/12

array(['20191111', '20191112', '20191113', '20191114', '20191115',
       '20191116', '20191117', '20191118', '20191119', '20191120',
       '20191121', '20191122', '20191123', '20191124', '20191125',
       '20191126', '20191127', '20191128', '20191129', '20191130',
       '20191201', '20191202', '20191203', '20191204', '20191205',
       '20191206', '20191207', '20191208', '20191209', '20191210',
       '20191211', '20191212'], dtype='<U8')

In [284]:
# channel_prog_cat_dict = {"|".join(x.split("|")[0:2]):x.split("|")[2] for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index}
# safe to assume same channel_prog_cat triples?


channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
programmes = ['5 news', '5 news at 5', '5 news lunchtime', '5 news tonight', '5 news weekend', '@ breakfast', 'a week in interviews', 'afternoon live', 'all out politics', 'ask the leaders', 'bbc london', 'bbc news', 'bbc news at 9', 'bbc news at five', 'bbc news at nine', 'bbc news at one', 'bbc news at six', 'bbc news at ten', 'bbc newsroom live', 'beyond 100 days', 'breakfast', 'brexitcast', "britain's care scandal exposed", 'channel 4 news', 'channel 4 news climate debate', 'channel 4 news summary', 'click', 'dateline london', 'election 2019: scotland', 'electioncast', 'fox news @ 9', 'good morning britain', 'ian king live', 'itv evening news', 'itv lunchtime news', 'itv news', 'itv news at ten', 'jeremy vine', 'johnson v corbyn: the itv debate', 'kay burley @breakfast', 'live: britain decides - the everything but brexit debate', 'loose women', 'lorraine', 'lunchtime live', 'macedonia: more than a name', 'mock the week', 'news on the hour', 'newsnight', 'newswatch', 'off air', 'on assignment', 'our world', 'outside source', 'panorama', 'party election broadcast', 'party election broadcast by the brexit party', 'party election broadcast by the conservative party', 'party election broadcast by the green party', 'party election broadcast by the labour party', 'party election broadcast by the liberal democrats', 'party election broadcast on behalf of plaid cymru', 'party election broadcast on behalf of the brexit party', 'party election broadcast on behalf of the conservative party', 'party election broadcast on behalf of the green party', 'party election broadcast on behalf of the labour party', 'party election broadcast on behalf of the liberal democrats', 'party election broadcast on behalf of the scottish national party', 'path to power', 'peston', "pmq's", 'politics live', 'press preview', 'question time', 'question time election special - under 30s', 'question time leaders special', 'regional news and weather', "rivals - america's endgame", 'sky news', 'sky news @ breakfast', 'sky news at 11', 'sky news at 11 with anna botting', 'sky news at six', 'sky news at ten', 'sky news including fyi', 'sky news tonight', 'sky news with sarah hewson', 'sophy ridge on sunday', 'sophy ridge: the take', 'sportsday', 'sportsline', 'talking movies', 'the andrew neil interviews', 'the andrew neil show', 'the bbc election debate', 'the campaign', 'the early rundown', 'the film review', 'the firing line', 'the itv election debate', 'the itv election interviews', 'the leader interviews: boris johnson - tonight', 'the leader interviews: jeremy corbyn - tonight', 'the leader interviews: jo swinson - tonight', 'the leader interviews: nigel farage - tonight', 'the leader interviews: tonight', 'the news hour with mark austin', 'the one show', 'the papers', 'the pledge', 'the prime ministerial debate', 'the prince and the epstein scandal - panorama', 'the real derry girls', 'the richard dimbleby lecture 2019', 'the sarah-jane mee show', 'the travel show', 'through the lens: untold stories', 'turner prize', 'universal credit: one year on - panorama', 'victoria derbyshire', 'victoria derbyshire: your election', 'witness history', 'worklife', 'world news today', 'your questions answered', 'your questions answered: green party', 'your questions answered: nigel farage']
categories = ["comedy","current affairs: politics","daytime","documentary: current affairs","interests","news"]
# 

In [ ]:
channel_prog_pairs = ["|".join(x.split("|")[0:2]) for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index]
categories = [x.split("|")[2] for x in ser.apply(lambda x: "|".join(x.split("|")[1:4]))[ser.apply(lambda x:len(x.split("|")))==4].value_counts().sort_index().index]
len(channel_prog_pairs),len(categories),

pd.DataFrame([channel_prog_pairs,categories]).T.groupby(0).sum().to_dict()#[1].values#[1].values

# sadly not a one-for-one channel-prog-prairto category mapping

In [258]:
np.unique(list(channel_prog_cat_dict.values()))

array(['comedy', 'current affairs', 'current affairs: politics',
       'daytime', 'documentary', 'documentary: current affairs',
       'interests', 'news'], dtype='<U28')

In [220]:
q1_tv_channels = pd.Series([x.split("|")[1] for x in ser.value_counts().loc[ser.value_counts().index[sec_no>=2]].index])
q1_tv_channels.value_counts()

bbcnews24    788
bbc1         760
itv1         629
skynews      567
bbc2         169
channel 4    129
channel 5    119
              22
it            16
bb            15
b             13
bbc           12
i              6
bbcne          5
channel        3
skyn           3
sk             3
ch             2
chan           2
bbcn           2
s              2
bbcnew         1
sky            1
c              1
skynew         1
itv            1
channe         1
chann          1
bbcnews        1
dtype: int64

In [298]:
for x in q1_tv_channels.value_counts().index:
    if x in channels:
        print(x)
    elif sum([x == y[0:len(x)] for y in channels])==1:
        x = channels[[y[0:len(x)] for y in channels].index(x)]
        print(x)
    else:
        x = np.nan
        print(x)
        

bbcnews24
bbc1
itv1
skynews
bbc2
channel 4
channel 5
nan
itv1
nan
nan
nan
itv1
bbcnews24
nan
skynews
skynews
nan
nan
bbcnews24
skynews
bbcnews24
skynews
nan
skynews
itv1
nan
nan
bbcnews24


In [297]:
ser = pd.Series( flatten( df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan).stack().values ) )
ser.value_counts()



'bbcnews24'

In [317]:
df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).max()

q1_before_yesterdayW18         7
q1_before_yesterday_newsW18    7
q1_yesterdayW18                7
q1_yesterday_newsW18           8
q1_todayW18                    7
q1_today_newsW18               7
dtype: int64

In [ ]:
### start building q1 dataframe

In [950]:
q1_base_cols

['q1_before_yesterdayW18_1',
 'q1_before_yesterdayW18_2',
 'q1_before_yesterdayW18_3',
 'q1_before_yesterdayW18_4',
 'q1_before_yesterdayW18_5',
 'q1_before_yesterdayW18_6',
 'q1_before_yesterdayW18_7',
 'q1_before_yesterday_newsW18_1',
 'q1_before_yesterday_newsW18_2',
 'q1_before_yesterday_newsW18_3',
 'q1_before_yesterday_newsW18_4',
 'q1_before_yesterday_newsW18_5',
 'q1_before_yesterday_newsW18_6',
 'q1_before_yesterday_newsW18_7',
 'q1_yesterdayW18_1',
 'q1_yesterdayW18_2',
 'q1_yesterdayW18_3',
 'q1_yesterdayW18_4',
 'q1_yesterdayW18_5',
 'q1_yesterdayW18_6',
 'q1_yesterdayW18_7',
 'q1_yesterday_newsW18_1',
 'q1_yesterday_newsW18_2',
 'q1_yesterday_newsW18_3',
 'q1_yesterday_newsW18_4',
 'q1_yesterday_newsW18_5',
 'q1_yesterday_newsW18_6',
 'q1_yesterday_newsW18_7',
 'q1_yesterday_newsW18_8',
 'q1_todayW18_1',
 'q1_todayW18_2',
 'q1_todayW18_3',
 'q1_todayW18_4',
 'q1_todayW18_5',
 'q1_todayW18_6',
 'q1_todayW18_7',
 'q1_today_newsW18_1',
 'q1_today_newsW18_2',
 'q1_today_newsW18_3',
 'q1_today_newsW18_4',
 'q1_today_newsW18_5',
 'q1_today_newsW18_6',
 'q1_today_newsW18_7']

In [878]:
q1_col_maxes = df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: len(x.split("~")) if pd.notnull(x) else 0).max()
q1_base_cols = flatten([[x+"_"+str(y+1) for y in range(q1_col_maxes.loc[x])] for x in q1_col_maxes.index])

In [879]:
q1 = pd.DataFrame(columns = q1_base_cols)

temp = df[match(df,"q1",case_sensitive=False).index].replace("__na__",np.nan)\
           .applymap(lambda x: x.split("~") if pd.notnull(x) else np.nan)
for col in q1_col_maxes.index:
    for no in range(1,q1_col_maxes.loc[col]+1):
        q1[col+"_"+str(no)] = temp[col].apply(lambda x: x[no-1] if isinstance(x,list) and len(x)>=no else np.nan)

In [880]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[0] if pd.notnull(x) and len(x.split("|"))>=1 else np.nan)
temp.columns = [x+"_dt" for x in q1_base_cols]
q1[temp.columns]=temp

In [881]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[1] if pd.notnull(x) and len(x.split("|"))>=2 else np.nan)
temp.columns = [x+"_chan" for x in q1_base_cols]
q1[temp.columns]=temp

In [882]:
temp = q1[q1_base_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))>=3 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_cols]
q1[temp.columns]=temp

In [883]:
q1[[x+"_prog" for x in q1_base_cols]].stack().value_counts()

bbc1|regional news and weather                       14155
bbc1|bbc news                                        10114
bbc1|breakfast                                        8471
bbc1|bbc news at ten                                  6488
bbc1|bbc news at six                                  4958
                                                     ...  
bbc1|the prince and the epstein scandal - panor          1
itv1|the le                                              1
bbc1|party election broadcast by                         1
bbc1|the prince and the epstein scandal - panoram        1
bbc1|the prim                                            1
Length: 467, dtype: int64

In [884]:
temp = q1[q1_base_cols].applymap(lambda x: x.split("|")[3] if pd.notnull(x) and len(x.split("|"))>=4 else np.nan)
temp.columns = [x+"_cat" for x in q1_base_cols]
q1[temp.columns]=temp

In [885]:
q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

bbc1         55981
itv1         18786
skynews      11781
bbcnews24    11502
bbc2          4562
channel 4     4147
channel 5      712
                23
it              18
bb              16
b               13
bbc             12
i                6
bbcne            5
skyn             4
channel          4
sk               3
ch               2
s                2
bbcn             2
chan             2
c                1
itv              1
sky              1
chann            1
bbcnews          1
skynew           1
channe           1
bbcnew           1
dtype: int64

In [ ]:
# vc = q1[[x+"_dt" for x in q1_base_cols]].applymap(lambda x:fix_q1_date_string(x)).stack().value_counts()

In [886]:
# vc.loc[[x for x in vc.index if len(x)<=11]]

In [887]:
def fix_q1_date_string(x):
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif len(x)==12:
        # it's definitely fine
        return(x)
    elif len(x)==11:
        # it's maybe fine if it's a short 2400 thing - e.g. 600, 915
        # but if the 24 hour string starts with 0/1/2 more reasonable to assume evening than morning
        if x[8] in ["0","1","2"]:
            x=x+"0"
        return(x)
    elif len(x)==10:
        # technically, this could be short or long 24 hour strings
        # but I eyeballed all instances and saw it was only 'long' 24 hour strings
        # some of these should also actually be 15/45 mins past the hour ...
        # but effectively rounding down to the nearest hour seems like an acceptable fix
        return(x+"00")
    else:
        # bugs me to throw away partial information
        # but you'd need to have a good reason to have to add in the extra work
        # needed to handle partially complete datestrings
        return(np.nan)

In [888]:
q1[[x+"_dt" for x in q1_base_cols]] = q1[[x+"_dt" for x in q1_base_cols]].applymap(lambda x:fix_q1_date_string(x))

q1[[x+"_dt" for x in q1_base_cols]].stack().value_counts()

201911181800    782
201911141800    721
201912021800    720
201911291800    714
201911221900    713
               ... 
201912082030      1
201912052240      1
201911111800      1
201911142340      1
201912032130      1
Length: 992, dtype: int64

In [889]:

# q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

In [890]:
def fix_q1_channel_string(x):
    channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in channels:
        # it's definitely fine
        return(x)
    elif sum([x == y[0:len(x)] for y in channels])==1:
        # unique match on what there is to one of the channel strings
        return channels[[y[0:len(x)] for y in channels].index(x)]
    else:
        # in theory, we could match to broader classes (e.g. "bbc" and "channel 4/5")
        # breaks my heart to throw data away!
        # but that would inconsistent with treamtent of the date strings and create extra baggage of overlapping categories
        # waaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa!
        # maybe 40-50 instances
        return np.nan

In [891]:
q1[[x+"_chan" for x in q1_base_cols]] = q1[[x+"_chan" for x in q1_base_cols]].applymap(lambda x:fix_q1_channel_string(x))
q1[[x+"_chan" for x in q1_base_cols]].stack().value_counts()

bbc1         55981
itv1         18811
skynews      11792
bbcnews24    11511
bbc2          4562
channel 4     4147
channel 5      712
dtype: int64

In [892]:
# q1[[x+"_cat" for x in q1_base_cols]].stack().value_counts()

In [893]:
def fix_q1_news_cat_string(x):
#     channels = ["bbc1","bbc2","bbcnews24","channel 4","channel 5","itv1","skynews"]
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    else:
        # anything other than empty cell should be "news"
        # at least, going on the distribution I see in the data
        # arguably, this makes this column unnecessary
        return "news"


In [898]:
q1_base_news_cols = [x for x in q1_base_cols if "news" in x]

q1[[x+"_cat" for x in q1_base_news_cols]] = q1[[x+"_cat" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_cat_string(x))

q1[[x+"_cat" for x in q1_base_news_cols]].stack().value_counts()

news    23152
dtype: int64

In [896]:
q1_categories = ['news', 'daytime', 'current affairs','current affairs: politics', 'interests',
        'documentary','documentary: current affairs', 'comedy', ]
q1_base_notnews_cols = [x for x in q1_base_cols if "news" not in x]
q1[[x+"_cat" for x in q1_base_notnews_cols]].stack().value_counts()


news                            49975
daytime                         24177
current affairs: politics        4943
interests                        1336
current affairs                  1223
documentary: current affairs      742
documentary                       368
comedy                             66
new                                57
                                   36
ne                                 25
n                                  24
current af                         24
intere                             23
current a                          20
curr                               19
current affairs:                   19
curre                              15
current affa                       14
cu                                 14
current affairs: p                 13
current affai                      11
current affairs: politi            10
current                             9
current affair                      9
current affairs: poli               9
day         

In [897]:
def fix_q1_notnews_cat_string(x):
    q1_categories = ['news', 'daytime', 'current affairs','current affairs: politics', 'interests',
        'documentary','documentary: current affairs', 'comedy', ]
    # sorted by generality/frequency
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_categories:
        # it's definitely fine
        return(x)
    elif sum([x == y[0:len(x)] for y in q1_categories])>=1:
        # since this data is somewhat subjective, I'm allowing 'greedy matches'
        # e.g. 'docu' -> 'documentary', 'c' -> 'current affairs'
        return q1_categories[[y[0:len(x)] for y in q1_categories].index(x)]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [899]:
q1[[x+"_cat" for x in q1_base_notnews_cols]] = q1[[x+"_cat" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_cat_string(x))
q1[[x+"_cat" for x in q1_base_notnews_cols]].stack().value_counts()

news                            50117
daytime                         24206
current affairs: politics        5015
current affairs                  1381
interests                        1373
documentary: current affairs      751
documentary                       382
comedy                             66
dtype: int64

In [900]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()

bbc1|regional news and weather    14155
bbc1|bbc news                     10114
bbc1|breakfast                     8471
bbc1|bbc news at ten               6488
bbc1|bbc news at six               4958
                                  ...  
bbc1|ask the leader                   1
itv1|party election broad             1
itv1|r                                1
itv1|the leader interv                1
bbc1|the prime                        1
Length: 337, dtype: int64

In [901]:
temp = q1[q1_base_notnews_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))==4 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_notnews_cols]
channel_prog_pairs_notnews_vc = temp.stack().value_counts()
channel_prog_pairs_notnews_vc
# 79

bbc1|regional news and weather                                                 14144
bbc1|bbc news                                                                  10108
bbc1|breakfast                                                                  8471
bbc1|bbc news at ten                                                            6484
bbc1|bbc news at six                                                            4956
                                                                               ...  
channel 5|party election broadcast on behalf of the green party                    9
bbc2|your questions answered: nigel farage                                         8
bbc2|your questions answered: green party                                          8
channel 5|party election broadcast on behalf of the scottish national party        7
channel 5|party election broadcast on behalf of the brexit party                   5
Length: 79, dtype: int64

In [902]:
temp = q1[q1_base_news_cols].applymap(lambda x: "|".join(x.split("|")[1:3]) if pd.notnull(x) and len(x.split("|"))==4 else np.nan)
temp.columns = [x+"_prog" for x in q1_base_news_cols]
channel_prog_pairs_news_vc = temp.stack().value_counts()
channel_prog_pairs_news_vc
# 63

bbcnews24|bbc news                            3034
skynews|press preview                         1858
skynews|all out politics                      1335
bbcnews24|the papers                          1301
skynews|sky news                              1245
                                              ... 
bbcnews24|rivals - america's endgame             2
bbcnews24|through the lens: untold stories       2
bbcnews24|off air                                2
bbcnews24|the firing line                        1
bbcnews24|turner prize                           1
Length: 63, dtype: int64

In [903]:
def fix_q1_notnews_chan_prog_string(x):
    q1_chan_prog_pairs = ['bbc1|regional news and weather', 'bbc1|bbc news', 'bbc1|breakfast', 'bbc1|bbc news at ten', 'bbc1|bbc news at six', 'itv1|regional news and weather', 'bbc1|bbc news at one', 'itv1|itv evening news', 'channel 4|channel 4 news', 'itv1|good morning britain', 'itv1|itv news', 'bbc2|newsnight', 'bbc1|the one show', 'bbc2|politics live', 'bbc1|the andrew neil interviews', 'itv1|itv lunchtime news', 'itv1|itv news at ten', 'bbc1|question time', 'bbc1|question time leaders special', 'itv1|johnson v corbyn: the itv debate', 'itv1|loose women', 'itv1|lorraine', 'bbc1|the prime ministerial debate', 'bbc2|bbc newsroom live', 'bbc2|victoria derbyshire', 'channel 4|party election broadcast', 'bbc1|the bbc election debate', 'channel 4|channel 4 news summary', 'channel 5|jeremy vine', 'itv1|the itv election debate', 'bbc1|party election broadcast by the labour party', 'bbc1|party election broadcast by the conservative party', 'bbc1|question time election special - under 30s', 'itv1|party election broadcast by the liberal democrats', 'itv1|party election broadcast by the labour party', 'bbc1|party election broadcast by the liberal democrats', 'itv1|party election broadcast by the brexit party', 'itv1|party election broadcast by the conservative party', 'bbc1|brexitcast', 'bbc2|bbc news at nine', 'bbc1|the prince and the epstein scandal - panorama', 'channel 5|5 news at 5', 'itv1|the leader interviews: jeremy corbyn - tonight', 'itv1|party election broadcast by the green party', 'itv1|the itv election interviews', 'itv1|the leader interviews: boris johnson - tonight', 'itv1|peston', 'bbc2|the andrew neil show', 'bbc2|bbc news at 9', 'itv1|the leader interviews: nigel farage - tonight', 'channel 4|channel 4 news climate debate', 'channel 5|5 news tonight', 'itv1|the leader interviews: tonight', 'bbc1|party election broadcast by the green party', 'itv1|the leader interviews: jo swinson - tonight', 'bbc1|party election broadcast by the brexit party', 'channel 5|5 news weekend', 'bbc1|panorama', 'bbc2|mock the week', 'bbc1|ask the leaders', 'channel 4|live: britain decides - the everything but brexit debate', 'bbc2|the one show', 'bbc1|the richard dimbleby lecture 2019', 'channel 5|5 news lunchtime', 'bbc1|universal credit: one year on - panorama', 'bbc1|bbc london', 'channel 5|5 news', 'bbc2|victoria derbyshire: your election', 'channel 5|party election broadcast on behalf of plaid cymru', 'channel 5|party election broadcast on behalf of the labour party', 'bbc2|your questions answered', 'channel 5|party election broadcast on behalf of the conservative party', 'itv1|on assignment', 'channel 5|party election broadcast on behalf of the liberal democrats', 'channel 5|party election broadcast on behalf of the green party', 'bbc2|your questions answered: nigel farage', 'bbc2|your questions answered: green party', 'channel 5|party election broadcast on behalf of the scottish national party', 'channel 5|party election broadcast on behalf of the brexit party']
    # sorted by frequency
    q1_chan_prog_generic_dict = {"itv1|the leader interviews: ":"the leader interviews(amb)",
                                 "bbc1|party election broadcast by the ":"party election broadcast(amb)",
                                 "itv1|party election broadcast by the ":"party election broadcast(amb)",
                                 "channel 5|party election broadcast on behalf of the ":"party election broadcast(amb)",
                                 "itv1|itv news":"itv news(amb)",
                                 'bbc1|bbc news at ':"bbc news(amb)",                                 
                                 'channel 4|channel 4 news':"channel 4 news(amb)",
                                 'channel 5|5 news':"channel 5 news(amb)",
                                 'bbc1|question time':'question time(amb)'}
    
    
    q1_chan_prog_generic_keys = list(q1_chan_prog_generic_dict.keys())
    # because I wept at the thought of throwing away so many ambiguous cases
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_chan_prog_pairs:
        # it's definitely fine
        return x.split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_pairs])==1:
        # I think we need to stick to unambiguous matches here
        # it is *theoretically* possible that we'd be able to disambiguate some of these by including the dates too!
        return q1_chan_prog_pairs[[y[0:len(x)] for y in q1_chan_prog_pairs].index(x)].split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_generic_keys])==1:
        # because 200+ ambiguous cases seemed too much -> got it down to 42
        return q1_chan_prog_generic_dict[q1_chan_prog_generic_keys[[y[0:len(x)] for y in q1_chan_prog_generic_keys].index(x)]]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [904]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x)).stack().value_counts()

regional news and weather                                            19357
bbc news                                                             10114
breakfast                                                             8471
bbc news at ten                                                       6504
bbc news at six                                                       4960
                                                                     ...  
your questions answered: nigel farage                                    8
party election broadcast on behalf of the scottish national party        7
party election broadcast on behalf of the brexit party                   5
question time(amb)                                                       4
channel 5 news(amb)                                                      2
Length: 79, dtype: int64

In [905]:
temp = q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x))
entries_dropped = (q1[[x+"_prog" for x in q1_base_notnews_cols]].stack(dropna=False).notnull() & temp.stack(dropna=False).isnull())
entries_dropped.sum()

28

In [907]:
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack(dropna=False)[entries_dropped].value_counts().head(60)

itv1|                    11
bbc1|                     7
bbc1|the                  2
bbc1|th                   2
bbc1|the p                1
itv1|the itv              1
itv1|the itv election     1
bbc2|                     1
bbc2|bbc new              1
itv1|the i                1
dtype: int64

In [909]:
q1[[x+"_prog" for x in q1_base_notnews_cols]] = q1[[x+"_prog" for x in q1_base_notnews_cols]].applymap(lambda x:fix_q1_notnews_chan_prog_string(x))
q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()

regional news and weather                                            19357
bbc news                                                             10114
breakfast                                                             8471
bbc news at ten                                                       6504
bbc news at six                                                       4960
                                                                     ...  
your questions answered: nigel farage                                    8
party election broadcast on behalf of the scottish national party        7
party election broadcast on behalf of the brexit party                   5
question time(amb)                                                       4
channel 5 news(amb)                                                      2
Length: 79, dtype: int64

In [908]:
# # woops!
# q1[[x+"_prog" for x in q1_base_notnews_cols]] = q1[[x+"_prog" for x in q1_base_notnews_cols]].replace("bbc1|question time(amb)","question time(amb)")
# q1[[x+"_prog" for x in q1_base_notnews_cols]].stack().value_counts()


In [910]:
channel_prog_pairs_news_vc

bbcnews24|bbc news                            3034
skynews|press preview                         1858
skynews|all out politics                      1335
bbcnews24|the papers                          1301
skynews|sky news                              1245
                                              ... 
bbcnews24|rivals - america's endgame             2
bbcnews24|through the lens: untold stories       2
bbcnews24|off air                                2
bbcnews24|the firing line                        1
bbcnews24|turner prize                           1
Length: 63, dtype: int64

In [911]:
# list(channel_prog_pairs_news_vc.index)

In [912]:
def fix_q1_news_chan_prog_string(x):
    q1_chan_prog_pairs = ['bbcnews24|bbc news', 'skynews|press preview', 'skynews|all out politics', 'bbcnews24|the papers', 'skynews|sky news', 'skynews|sky news at ten', 'bbcnews24|breakfast', 'skynews|kay burley @breakfast', 'bbcnews24|bbc news at six', 'skynews|sky news tonight', 'bbcnews24|afternoon live', 'bbcnews24|bbc news at one', 'bbcnews24|bbc newsroom live', 'bbcnews24|bbc news at ten', 'skynews|sophy ridge on sunday', 'skynews|the sarah-jane mee show', 'bbcnews24|victoria derbyshire', 'bbcnews24|bbc news at nine', 'skynews|lunchtime live', 'bbcnews24|beyond 100 days', 'skynews|sky news @ breakfast', 'skynews|the early rundown', 'bbcnews24|bbc news at five', 'skynews|ian king live', 'skynews|sky news at six', 'skynews|the campaign', 'bbcnews24|sportsday', 'bbcnews24|electioncast', 'skynews|sky news at 11', 'skynews|path to power', 'skynews|@ breakfast', 'skynews|the news hour with mark austin', 'bbcnews24|outside source', 'bbcnews24|world news today', 'bbcnews24|worklife', 'skynews|sophy ridge: the take', 'skynews|sky news with sarah hewson', 'bbcnews24|dateline london', 'bbcnews24|click', 'skynews|sky news including fyi', 'skynews|a week in interviews', 'skynews|sky news at 11 with anna botting', 'bbcnews24|the film review', 'bbcnews24|brexitcast', 'bbcnews24|the travel show', 'skynews|news on the hour', 'skynews|sportsline', 'skynews|the pledge', 'bbcnews24|newswatch', 'skynews|fox news @ 9', 'bbcnews24|our world', "skynews|pmq's", 'bbcnews24|talking movies', 'bbcnews24|witness history', 'bbcnews24|election 2019: scotland', "bbcnews24|britain's care scandal exposed", 'bbcnews24|the real derry girls', 'bbcnews24|macedonia: more than a name', "bbcnews24|rivals - america's endgame", 'bbcnews24|through the lens: untold stories', 'bbcnews24|off air', 'bbcnews24|the firing line', 'bbcnews24|turner prize']
    # sorted by frequency
    q1_chan_prog_generic_dict = {"bbcnews24|bbc news at ":"bbc news(amb)",
                                 "skynews|sky news at ":"sky news(amb)",
                                }
    
    q1_chan_prog_generic_keys = list(q1_chan_prog_generic_dict.keys())
    # because I wept at the thought of throwing away so many ambiguous cases
    if pd.isnull(x):
        # it's a nan, nothing to see here
        return x
    elif x==" ":
        return np.nan
    elif x in q1_chan_prog_pairs:
        # it's definitely fine
        return x.split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_pairs])==1:
        # I think we need to stick to unambiguous matches here
        # it is *theoretically* possible that we'd be able to disambiguate some of these by including the dates too!
        return q1_chan_prog_pairs[[y[0:len(x)] for y in q1_chan_prog_pairs].index(x)].split("|")[1]
    elif sum([x == y[0:len(x)] for y in q1_chan_prog_generic_keys])==1:
        # because 50+ ambiguous cases seemed too much -> got it down to 4
        return q1_chan_prog_generic_dict[q1_chan_prog_generic_keys[[y[0:len(x)] for y in q1_chan_prog_generic_keys].index(x)]]
    else:
        # should be pretty much nothing reaching here!
        return np.nan

In [913]:
q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x)).stack().value_counts()

bbc news                            3037
press preview                       1871
all out politics                    1336
the papers                          1306
sky news                            1247
                                    ... 
through the lens: untold stories       2
off air                                2
rivals - america's endgame             2
turner prize                           1
the firing line                        1
Length: 65, dtype: int64

In [914]:
temp = q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x))
entries_dropped = (q1[[x+"_prog" for x in q1_base_news_cols]].stack(dropna=False).notnull() & temp.stack(dropna=False).isnull())
entries_dropped.sum()

5

In [915]:
q1[[x+"_prog" for x in q1_base_news_cols]].stack(dropna=False)[entries_dropped].value_counts().head(60)

bbcnews24|th         2
bbcnews24|the        1
skynews|sophy rid    1
skynews|the          1
dtype: int64

In [916]:
q1[[x+"_prog" for x in q1_base_news_cols]] = q1[[x+"_prog" for x in q1_base_news_cols]].applymap(lambda x:fix_q1_news_chan_prog_string(x))
q1[[x+"_prog" for x in q1_base_news_cols]].stack().value_counts()

bbc news                            3037
press preview                       1871
all out politics                    1336
the papers                          1306
sky news                            1247
                                    ... 
through the lens: untold stories       2
off air                                2
rivals - america's endgame             2
turner prize                           1
the firing line                        1
Length: 65, dtype: int64

In [917]:
### q1 stuff ... almost done!

def turn_datestrings_to_dates(x):
    if pd.isnull(x):
        return pd.NaT
    elif len(x)==11:
        dstr = list(x)
        dstr.insert(8,"0")
        dstr = "".join(dstr)
        return pd.to_datetime(dstr,format='%Y%m%d%H%M')
    else:
        return pd.to_datetime(x,format='%Y%m%d%H%M')

        

In [918]:
# slowish ~ 1 min - could speed up by prepping columns and processing all at once
temp = q1[[x+"_dt" for x in q1_base_cols]].applymap( lambda x: turn_datestrings_to_dates(x) )

In [919]:
temp["q1_today_newsW18_7_dt"].value_counts().sort_index()

2019-11-17 23:00:00    1
2019-11-22 22:40:00    1
2019-11-23 22:30:00    1
2019-11-29 23:30:00    1
2019-11-30 17:00:00    1
2019-12-02 21:00:00    1
2019-12-08 13:00:00    1
2019-12-11 18:00:00    1
Name: q1_today_newsW18_7_dt, dtype: int64

In [923]:
q1[[x+"_dt" for x in q1_base_cols]] = q1[[x+"_dt" for x in q1_base_cols]].applymap( lambda x: turn_datestrings_to_dates(x) )

In [926]:
q1.dropna(how='all')

,q1_before_yesterdayW18_1,q1_before_yesterdayW18_2,q1_before_yesterdayW18_3,q1_before_yesterdayW18_4,q1_before_yesterdayW18_5,q1_before_yesterdayW18_6,q1_before_yesterdayW18_7,q1_before_yesterday_newsW18_1,q1_before_yesterday_newsW18_2,q1_before_yesterday_newsW18_3,...,q1_todayW18_5_cat,q1_todayW18_6_cat,q1_todayW18_7_cat,q1_today_newsW18_1_cat,q1_today_newsW18_2_cat,q1_today_newsW18_3_cat,q1_today_newsW18_4_cat,q1_today_newsW18_5_cat,q1_today_newsW18_6_cat,q1_today_newsW18_7_cat
id,,,,,,,,,,,,,,,,,,,,,
1,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98448,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98449,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [925]:
q1.dtypes.value_counts()

object            166
datetime64[ns]     43
float64             6
dtype: int64

In [927]:
# drop the completely blank 8th quadruple
q1.drop(['q1_yesterday_newsW18_8_dt', 'q1_yesterday_newsW18_8_chan',
       'q1_yesterday_newsW18_8_prog',  'q1_yesterday_newsW18_8_cat',  ],axis=1,inplace=True)

In [929]:
# what to do about these?
# can probably infer these *reasonably* based on most common channel:prog:cat triples


#'q1_before_yesterdayW18_7_cat', 'q1_yesterdayW18_7_cat', 'q1_todayW18_7_cat'

In [928]:
list( q1.columns[ q1.dtypes == 'float64' ] )

['q1_before_yesterdayW18_7_cat', 'q1_yesterdayW18_7_cat', 'q1_todayW18_7_cat']

In [932]:
q1.dtypes.value_counts()

object            166
datetime64[ns]     42
float64             3
dtype: int64

In [947]:
q1[q1.columns[ q1.dtypes!="datetime64[ns]" ]] = q1[q1.columns[ q1.dtypes!="datetime64[ns]" ]].replace(" ",np.nan).astype('category')

In [948]:
q1.dtypes.value_counts()

datetime64[ns]    42
category          21
category          21
category          18
category          13
                  ..
category           1
category           1
category           1
category           1
category           1
Length: 96, dtype: int64

In [ ]:
# replace(" ",np.nan)
# shared categories explicitly defined

In [949]:
q1.to_pickle('../BES_analysis_data/W19_comb_strings\\q1.zip', compression='zip')

In [23]:
## 

dataset_name = "W19_comb_strings"
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

import pandas as pd
import numpy as np
# save csv manually with SPSSS

csvfile = '../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.csv'
encoding = "ISO-8859-1"
df = pd.read_csv(csvfile,encoding=encoding)

df.rename(columns={'ï»¿id':'id'},inplace=True)
# bug screwing up character encoding of that file!

df = df.set_index("id").sort_index()
df = df.apply(lambda x: x.str.lower().replace(" ",np.nan).astype('category'))
df.to_pickle('../BES_analysis_data/W19_comb_strings\\BES2019_W19Strings_v0.2.zip', compression='zip')

# csv -> zip - loses all dtypes/structure ... but no interesting dtypes in the string dataset
